In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install ipywidgets
!pip install streamlit
from IPython.display import display, clear_output, Image, HTML
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 97.0 MB/s eta 0:00:00
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [ ]:
from sklearn.model_selection import train_test_split
import os, shutil, random

base_dir = '/content/drive/MyDrive/plastic_dataset'
os.makedirs(base_dir, exist_ok=True)

classes = ['plastic_bottle', 'plastic_container']
splits = ['train', 'val', 'test']

for split in splits:
    for cls in classes:
        os.makedirs(f'{base_dir}/{split}/{cls}', exist_ok=True)

# Move images automatically
src_dir = '/content/drive/MyDrive/plastic/dataset'
for cls in classes:
    imgs = os.listdir(f'{src_dir}/{cls}')
    random.shuffle(imgs)
    n_total = len(imgs)
    n_train = int(0.7 * n_total)
    n_val = int(0.2 * n_total)

    for i, img in enumerate(imgs):
        src_path = f'{src_dir}/{cls}/{img}'
        if i < n_train:
            dst = f'{base_dir}/train/{cls}/{img}'
        elif i < n_train + n_val:
            dst = f'{base_dir}/val/{cls}/{img}'
        else:
            dst = f'{base_dir}/test/{cls}/{img}'
        shutil.copy(src_path, dst)


In [ ]:
import os

base_dir = '/content/drive/MyDrive/plastic_dataset'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')


In [ ]:
import tensorflow as tf
import os

base_dir = '/content/drive/MyDrive/plastic_dataset'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')

img_size = (160, 160)
batch_size = 16

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=img_size,
    batch_size=batch_size
)

# ✅ Get class names BEFORE prefetching
class_names = train_ds.class_names
print("Classes found:", class_names)

# Optimize performance (AFTER reading class names)
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2

base_model = MobileNetV2(input_shape=(160, 160, 3),
                         include_top=False,
                         weights='imagenet')

base_model.trainable = False  # freeze pretrained layers

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)


In [ ]:
val_loss, val_acc = model.evaluate(val_ds)
print(f"\n✅ Validation Accuracy: {val_acc * 100:.2f}%")


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

img_path = "/content/drive/MyDrive/plastic/dataset/plastic_container/plastic_containers 366.jpg"  # change this
img = tf.keras.utils.load_img(img_path, target_size=(160, 160))
img_array = tf.keras.utils.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

pred = model.predict(img_array)
pred_class = class_names[np.argmax(pred)]
confidence = np.max(pred) * 100

print(f"🔍 Predicted: {pred_class} ({confidence:.2f}% confidence)")


In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

# --- Load datasets ---
train_ds = tf.keras.utils.image_dataset_from_directory(
    '/content/drive/MyDrive/plastic_dataset/train',   # change to your path
    image_size=(160, 160),
    batch_size=32
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    '/content/drive/MyDrive/plastic_dataset/val',     # change to your path
    image_size=(160, 160),
    batch_size=32
)

# ✅ Store class names BEFORE caching
class_names = train_ds.class_names
print("Classes found:", class_names)

# --- Optimize datasets ---
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
img_path = "/content/drive/MyDrive/test2.jpg"  # your test image
img = tf.keras.utils.load_img(img_path, target_size=(160, 160))
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) / 255.0

pred = model.predict(img_array)
predicted_class = class_names[np.argmax(pred)]
print("Predicted Class:", predicted_class)


In [ ]:
pred = model.predict(img_array)[0]
for i, score in enumerate(pred):
    print(f"{class_names[i]}: {score*100:.2f}%")

# Or top-3 only:
top3 = np.argsort(pred)[-3:][::-1]
for i in top3:
    print(f"{class_names[i]}: {pred[i]*100:.2f}%")


In [ ]:
from google.colab import files
from tensorflow.keras.preprocessing import image
import numpy as np
import tensorflow as tf

# Upload image
uploaded = files.upload()

for filename in uploaded.keys():
    img_path = filename
    img = tf.keras.utils.load_img(img_path, target_size=(160, 160))
    img_array = tf.keras.utils.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    # Predict
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    class_names = ["plastic_bottle", "plastic_container"]  # your class names
    predicted_class = class_names[np.argmax(score)]
    confidence = 100 * np.max(score)

    print(f"\n✅ Predicted Class: {predicted_class}")
    print(f"📊 Confidence: {confidence:.2f}%")


In [ ]:
from google.colab import files


In [ ]:
from google.colab import files
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

def classify_uploaded_image(model):
    uploaded = files.upload()
    if not uploaded:
        print("⚠️ No file uploaded.")
        return None

    for filename in uploaded.keys():
        img = Image.open(filename).convert("RGB")
        img_resized = img.resize((160, 160))
        img_array = np.array(img_resized) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        prediction = model.predict(img_array)
        predicted_class = np.argmax(prediction)

        # 🧠 Assuming: 0 = plastic_bottle, 1 = plastic_container
        material_type = "bottle" if predicted_class == 0 else "container"

        print(f"\n✅ Predicted Material: {material_type}")
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Predicted: {material_type}")
        plt.show()
        return material_type


In [ ]:
import os, base64
from IPython.display import display, HTML
from google.colab import output

def interactive_gallery(material_type, predicted_image_path=None):
    base_path = "/content/drive/MyDrive/restructure_material"
    folder = os.path.join(base_path, material_type.lower())

    if not os.path.exists(folder):
        print(f"⚠ Folder '{folder}' not found.")
        return

    images = [f for f in os.listdir(folder) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
    if not images:
        print("No images found.")
        return

    # ✅ Group by farming type
    categories = {
        "Soil-based Farming": [img for img in images if "soil" in img.lower()],
        "Hydroponics": [img for img in images if "hydro" in img.lower()],
        "Aeroponics": [img for img in images if "aero" in img.lower()],
    }

    # ✅ General instructions (5 points)
    general_instructions = [
        "♻️ Clean the waste material thoroughly before reuse.",
        "🪚 Cut or modify it safely using appropriate tools.",
        "🌱 Choose a farming method (Soil-based, Hydroponics, Aeroponics).",
        "💧 Maintain proper watering or nutrient supply depending on method.",
        "☀️ Place the setup under adequate light or temperature conditions."
    ]

    # ✅ 3-line instructions per category
    instructions = {
        "Soil-based Farming": [
            "1️⃣ Fill the restructured material with soil and compost mix.",
            "2️⃣ Insert the plant sapling and ensure proper drainage holes.",
            "3️⃣ Place it under sunlight and water regularly."
        ],
        "Hydroponics": [
            "1️⃣ Fill with nutrient solution up to one-third of the container.",
            "2️⃣ Support roots with a sponge or cocopeat medium.",
            "3️⃣ Maintain proper aeration and nutrient pH balance."
        ],
        "Aeroponics": [
            "1️⃣ Suspend roots in air and mist them periodically.",
            "2️⃣ Ensure nutrient spray nozzles are clean and functional.",
            "3️⃣ Maintain high humidity and stable temperature."
        ]
    }

    html = """
    <style>
      .gallery { display: flex; flex-wrap: wrap; gap: 10px; justify-content: flex-start; margin-left: 30px; }
      .thumb { width: 150px; height: 150px; object-fit: cover; cursor: pointer; border-radius: 10px; transition: transform 0.2s; }
      .thumb:hover { transform: scale(1.05); }
      .overlay {
        position: fixed; top: 0; left: 0; width: 100%; height: 100%;
        background: rgba(0,0,0,0.9); display: none; justify-content: center;
        align-items: center; flex-direction: column; z-index: 1000;
      }
      .overlay img { max-width: 85%; max-height: 85%; border-radius: 12px; }
      .select-btn {
        background-color: #4CAF50; color: white; border: none;
        padding: 12px 24px; font-size: 16px; border-radius: 8px;
        margin-top: 15px; cursor: pointer;
      }
      .select-btn:hover { background-color: #45a049; }
      .close-text { color: #ccc; margin-top: 10px; font-size: 14px; }
      h1, h2 { color: #2E8B57; text-align: left; margin-left: 30px; }
      ul { text-align: left; max-width: 800px; margin: 10px 30px; color: #444; }
      .pred-img { display: block; margin: 10px 30px; max-width: 250px; border-radius: 10px; border: 2px solid #4CAF50; }
    </style>
    """

    # ✅ Display predicted image (if available)
    if predicted_image_path and os.path.exists(predicted_image_path):
        with open(predicted_image_path, "rb") as img_file:
            b64_pred = base64.b64encode(img_file.read()).decode("utf-8")
        mime = "image/jpeg" if predicted_image_path.lower().endswith(".jpg") else "image/png"
        html += f"<h1>Predicted Material: {material_type.capitalize()}</h1>"
        html += f"<img src='data:{mime};base64,{b64_pred}' class='pred-img'>"

    # ✅ General instructions
    html += "<h2>🧭 General Instructions for Reuse</h2><ul>"
    for line in general_instructions:
        html += f"<li>{line}</li>"
    html += "</ul><hr>"

    # ✅ Build HTML for each category
    for cat, imgs in categories.items():
        if not imgs:
            continue
        html += f"<h2>{cat}</h2><ul>"
        for line in instructions[cat]:
            html += f"<li>{line}</li>"
        html += "</ul><div class='gallery'>"

        for img_name in imgs:
            img_path = os.path.join(folder, img_name)
            with open(img_path, "rb") as img_file:
                b64 = base64.b64encode(img_file.read()).decode("utf-8")
            mime = "image/jpeg" if img_name.lower().endswith(".jpg") else "image/png"
            html += f"<img src='data:{mime};base64,{b64}' class='thumb' onclick='showImage(this.src, \"{img_name}\")'>"

        html += "</div><hr>"

    # ✅ Overlay and JavaScript
    html += """
    <div class="overlay" id="overlay" onclick="hideOverlay(event)">
      <img id="overlay-img" src="">
      <button class="select-btn" id="selectBtn" onclick="selectImage()">Select</button>
      <div class="close-text">(Click anywhere or press Esc to close)</div>
    </div>

    <script>
      let selectedImageName = "";
      function showImage(src, name) {
        const overlay = document.getElementById("overlay");
        const img = document.getElementById("overlay-img");
        img.src = src;
        selectedImageName = name;
        overlay.style.display = "flex";
      }
      function hideOverlay(event) {
        if (event.target.id === "overlay" || event.key === "Escape") {
          document.getElementById("overlay").style.display = "none";
        }
      }
      function selectImage() {
        google.colab.kernel.invokeFunction('notebook.selectedImageCallback', [selectedImageName], {});
        document.getElementById("overlay").style.display = "none";
      }
      document.addEventListener('keydown', (e) => {
        if (e.key === "Escape") hideOverlay(e);
      });
    </script>
    """

    display(HTML(html))

# ✅ Callback registration
selected_image = None

def handle_selected_image(name):
    global selected_image
    selected_image = name
    print(f"✅ Selected image: {name}")
    while selected_image is None:
        sleep(0.5)  # Wait for user to select
    return selected_image

output.register_callback('notebook.selectedImageCallback', handle_selected_image)


In [ ]:
# Suppose your model predicts material_type
material_type =classify_uploaded_image(model)
interactive_gallery(material_type)



In [ ]:
model.save("/content/drive/MyDrive/plastic_classifier_model.h5")


In [ ]:
model.save("/content/drive/MyDrive/plastic_classifier_model.keras")


In [ ]:
from tensorflow import keras

# Load model
model = keras.models.load_model("/content/drive/MyDrive/plastic_classifier_model.keras")

print("✅ Model loaded successfully!")


✅ Model loaded successfully!


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 4 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
from tensorflow import keras

# Load model
model = keras.models.load_model("/content/drive/MyDrive/plastic_classifier_model.h5")

print("✅ Model loaded successfully!")


✅ Model loaded successfully!


In [ ]:
!pip install gradio --quiet
import gradio as gr
import os
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model

# ---------------- CONFIG ----------------
MODEL_PATH = "/content/drive/MyDrive/plastic_classifier_model.h5"
BASE_PATH = "/content/drive/MyDrive/restructure_material"

# ---------------- MODEL ----------------
try:
    model = load_model(MODEL_PATH, compile=False)
    print("✅ Model loaded successfully!")
except Exception as e:
    print("⚠️ Model load failed, using dummy:", e)
    class Dummy:
        def predict(self, x): return np.array([[0.9, 0.1]])
    model = Dummy()

# ---------------- HELPERS ----------------
def classify_uploaded_image(uploaded_path):
    if not uploaded_path:
        return "⚠️ Upload an image first.", None
    img = Image.open(uploaded_path).convert("RGB").resize((160,160))
    arr = np.expand_dims(np.array(img)/255., axis=0)
    pred = model.predict(arr)
    pred = np.array(pred).squeeze()
    if pred.size == 1:
        idx = 0 if pred < 0.5 else 1
        conf = float(pred)*100
    else:
        idx = int(np.argmax(pred))
        conf = float(np.max(pred))*100
    material = "bottle" if idx==0 else "container"
    return f"✅ Predicted Material: {material.upper()}", material

def load_category_images(material):
    folder = os.path.join(BASE_PATH, material.lower())
    if not os.path.exists(folder):
        return [], [], []
    files = [f for f in os.listdir(folder) if f.lower().endswith(('.jpg','.jpeg','.png'))]
    soil = [os.path.join(folder,f) for f in files if 'soil' in f.lower()]
    hydro = [os.path.join(folder,f) for f in files if 'hydro' in f.lower()]
    aero = [os.path.join(folder,f) for f in files if 'aero' in f.lower()]
    return soil, hydro, aero

def get_instructions(material):
    general_instructions = """
### 🧭 General Instructions
- ♻️ Clean the waste material thoroughly before reuse.
- 🪚 Cut or modify safely using proper tools.
- 🌱 Choose a farming method (Soil-based, Hydroponics, Aeroponics).
- 💧 Maintain correct watering or nutrient levels.
- ☀️ Provide adequate sunlight or controlled temperature.
"""

    soil_instructions = """
### 🌱 Soil-based Farming
1. Fill with soil and compost mix.
2. Add sapling and ensure drainage holes.
3. Place under sunlight and water regularly.
"""

    hydro_instructions = """
### 💧 Hydroponics
1. Fill partially with nutrient solution.
2. Support roots using cocopeat or sponge.
3. Maintain aeration and pH balance.
"""

    aero_instructions = """
### 🌬️ Aeroponics
1. Suspend roots in air and mist periodically.
2. Keep spray nozzles clean.
3. Maintain humidity and temperature.
"""

    return general_instructions, soil_instructions, hydro_instructions, aero_instructions

# ---------------- CALLBACKS ----------------
def on_upload(img_path):
    status, material = classify_uploaded_image(img_path)
    general, soil_inst, hydro_inst, aero_inst = get_instructions(material)

    if material is None:
        return status, general, "", "", "", [], [], [], "No selection yet."

    soil, hydro, aero = load_category_images(material)

    def to_gallery(paths):
        return [(p, os.path.basename(p)) for p in paths if os.path.exists(p)]

    return status, general, soil_inst, hydro_inst, aero_inst, to_gallery(soil), to_gallery(hydro), to_gallery(aero), "No selection yet."


# ✅ FIXED SELECTION HANDLER
def on_gallery_select(evt: gr.SelectData, gallery_data):
    """Handle gallery selection with proper error handling"""
    try:
        selected_index = evt.index
        if gallery_data is None or len(gallery_data) == 0:
            return "⚠️ No images in gallery"

        if selected_index >= len(gallery_data):
            return "⚠️ Invalid selection"

        selected_item = gallery_data[selected_index]  # (filepath, label)
        selected_path = selected_item[0]
        selected_name = os.path.basename(selected_path)
        return selected_name  # Store just the filename
    except Exception as e:
        return f"⚠️ Error: {str(e)}"


def confirm_selected(selected_name):
    """Confirm the selection when button is clicked"""
    if not selected_name or "⚠️" in selected_name or "No selection" in selected_name:
        return "⚠️ No design selected. Click a thumbnail first."
    return f"✅ You selected: **{selected_name}**"

# ---------------- UI ----------------
with gr.Blocks(theme=gr.themes.Soft(primary_hue="green")) as app:
    gr.Markdown(
        """
        # ♻️ Smart Waste Restructure System
        Upload an image of a **plastic waste** item to classify it (bottle/container).
        Then explore eco-friendly **reuse ideas** under Soil, Hydro, and Aeroponics 🌱
        Click any image to select it.
        """
    )

    with gr.Row():
        upload = gr.Image(label="📤 Upload Material", type="filepath", height=160)
        status = gr.Textbox(label="Prediction", interactive=False)

    general_box = gr.Markdown("Upload an image to get started.")

    with gr.Row():
        with gr.Tabs():
            with gr.TabItem("Soil-based Farming"):
                soil_inst_box = gr.Markdown("")
                soil_gallery = gr.Gallery(columns=4, height=220)
            with gr.TabItem("Hydroponics"):
                hydro_inst_box = gr.Markdown("")
                hydro_gallery = gr.Gallery(columns=4, height=220)
            with gr.TabItem("Aeroponics"):
                aero_inst_box = gr.Markdown("")
                aero_gallery = gr.Gallery(columns=4, height=220)

    with gr.Row():
        choose_btn = gr.Button("Select this design")
        selected_text = gr.Markdown("No selection yet.")

    # Wiring with proper state management
    upload.change(
        on_upload,
        inputs=upload,
        outputs=[status, general_box, soil_inst_box, hydro_inst_box, aero_inst_box,
                 soil_gallery, hydro_gallery, aero_gallery, selected_text]
    )

    # Pass gallery component as input to access its current value
    soil_gallery.select(
        on_gallery_select,
        inputs=[soil_gallery],
        outputs=[selected_text]
    )

    hydro_gallery.select(
        on_gallery_select,
        inputs=[hydro_gallery],
        outputs=[selected_text]
    )

    aero_gallery.select(
        on_gallery_select,
        inputs=[aero_gallery],
        outputs=[selected_text]
    )

    choose_btn.click(confirm_selected, inputs=selected_text, outputs=selected_text)

app.launch(share=True)

In [ ]:
!pip install gradio --quiet
import gradio as gr
import os
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model

# ---------------- CONFIG ----------------
MODEL_PATH = "/content/drive/MyDrive/plastic_classifier_model.h5"
BASE_PATH = "/content/drive/MyDrive/restructure_material"

# ---------------- MODEL ----------------
try:
    model = load_model(MODEL_PATH, compile=False)
    print("✅ Model loaded successfully!")
except Exception as e:
    print("⚠️ Model load failed, using dummy:", e)
    class Dummy:
        def predict(self, x): return np.array([[0.9, 0.1]])
    model = Dummy()

# ---------------- HELPERS ----------------
def classify_uploaded_image(uploaded_path):
    if not uploaded_path:
        return "⚠️ Upload an image first.", None
    img = Image.open(uploaded_path).convert("RGB").resize((160,160))
    arr = np.expand_dims(np.array(img)/255., axis=0)
    pred = model.predict(arr)
    pred = np.array(pred).squeeze()
    if pred.size == 1:
        idx = 0 if pred < 0.5 else 1
        conf = float(pred)*100
    else:
        idx = int(np.argmax(pred))
        conf = float(np.max(pred))*100
    material = "bottle" if idx==0 else "container"
    return f"✅ Predicted Material: {material.upper()}", material

def load_category_images(material):
    folder = os.path.join(BASE_PATH, material.lower())
    if not os.path.exists(folder):
        return [], [], []
    files = [f for f in os.listdir(folder) if f.lower().endswith(('.jpg','.jpeg','.png'))]
    soil = [os.path.join(folder,f) for f in files if 'soil' in f.lower()]
    hydro = [os.path.join(folder,f) for f in files if 'hydro' in f.lower()]
    aero = [os.path.join(folder,f) for f in files if 'aero' in f.lower()]
    return soil, hydro, aero

def get_instructions(material):
    general_instructions = """
### 🧭 General Instructions
- ♻️ Clean the waste material thoroughly before reuse.
- 🪚 Cut or modify safely using proper tools.
- 🌱 Choose a farming method (Soil-based, Hydroponics, Aeroponics).
- 💧 Maintain correct watering or nutrient levels.
- ☀️ Provide adequate sunlight or controlled temperature.
"""

    soil_instructions = """
### 🌱 Soil-based Farming
1. Fill with soil and compost mix.
2. Add sapling and ensure drainage holes.
3. Place under sunlight and water regularly.
"""

    hydro_instructions = """
### 💧 Hydroponics
1. Fill partially with nutrient solution.
2. Support roots using cocopeat or sponge.
3. Maintain aeration and pH balance.
"""

    aero_instructions = """
### 🌬️ Aeroponics
1. Suspend roots in air and mist periodically.
2. Keep spray nozzles clean.
3. Maintain humidity and temperature.
"""

    return general_instructions, soil_instructions, hydro_instructions, aero_instructions

# ---------------- CALLBACKS ----------------
def on_upload(img_path):
    status, material = classify_uploaded_image(img_path)
    general, soil_inst, hydro_inst, aero_inst = get_instructions(material)

    if material is None:
        return status, general, "", "", "", [], [], [], "No selection yet."

    soil, hydro, aero = load_category_images(material)

    def to_gallery(paths):
        return [(p, os.path.basename(p)) for p in paths if os.path.exists(p)]

    return status, general, soil_inst, hydro_inst, aero_inst, to_gallery(soil), to_gallery(hydro), to_gallery(aero), "No selection yet."


# ✅ FIXED SELECTION HANDLER
def on_gallery_select(evt: gr.SelectData, gallery_data):
    """Handle gallery selection with proper error handling"""
    try:
        selected_index = evt.index
        if gallery_data is None or len(gallery_data) == 0:
            return "⚠️ No images in gallery"

        if selected_index >= len(gallery_data):
            return "⚠️ Invalid selection"

        selected_item = gallery_data[selected_index]  # (filepath, label)
        selected_path = selected_item[0]
        selected_name = os.path.basename(selected_path)
        return selected_name  # Store just the filename
    except Exception as e:
        return f"⚠️ Error: {str(e)}"


def confirm_selected(selected_name):
    """Confirm the selection when button is clicked"""
    if not selected_name or "⚠️" in selected_name or "No selection" in selected_name:
        return "⚠️ No design selected. Click a thumbnail first."
    return f"✅ You selected: **{selected_name}**"

# ---------------- UI ----------------
with gr.Blocks(theme=gr.themes.Soft(primary_hue="green")) as app:
    gr.Markdown(
        """
        # ♻️ Smart Waste Restructure System
        Upload an image of a **plastic waste** item to classify it (bottle/container).
        Then explore eco-friendly **reuse ideas** under Soil, Hydro, and Aeroponics 🌱
        Click any image to select it.
        """
    )

    with gr.Row():
        upload = gr.Image(label="📤 Upload Material", type="filepath", height=160)
        status = gr.Textbox(label="Prediction", interactive=False)

    general_box = gr.Markdown("Upload an image to get started.")

    # Soil-based Farming Section
    gr.Markdown("---")
    soil_inst_box = gr.Markdown("")
    soil_gallery = gr.Gallery(columns=6, height=150, label="Soil-based Farming", object_fit="cover", container=False, show_label=True)

    # Hydroponics Section
    gr.Markdown("---")
    hydro_inst_box = gr.Markdown("")
    hydro_gallery = gr.Gallery(columns=6, height=150, label="Hydroponics", object_fit="cover", container=False, show_label=True)

    # Aeroponics Section
    gr.Markdown("---")
    aero_inst_box = gr.Markdown("")
    aero_gallery = gr.Gallery(columns=6, height=150, label="Aeroponics", object_fit="cover", container=False, show_label=True)
    gr.Markdown("---")

    with gr.Row():
        choose_btn = gr.Button("Select this design")
        selected_text = gr.Markdown("No selection yet.")

    # Wiring with proper state management
    upload.change(
        on_upload,
        inputs=upload,
        outputs=[status, general_box, soil_inst_box, hydro_inst_box, aero_inst_box,
                 soil_gallery, hydro_gallery, aero_gallery, selected_text]
    )

    # Pass gallery component as input to access its current value
    soil_gallery.select(
        on_gallery_select,
        inputs=[soil_gallery],
        outputs=[selected_text]
    )

    hydro_gallery.select(
        on_gallery_select,
        inputs=[hydro_gallery],
        outputs=[selected_text]
    )

    aero_gallery.select(
        on_gallery_select,
        inputs=[aero_gallery],
        outputs=[selected_text]
    )

    choose_btn.click(confirm_selected, inputs=selected_text, outputs=selected_text)

app.launch(share=True)

In [ ]:
!pip install gradio --quiet
import gradio as gr
import os
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model
import random

# ---------------- CONFIG ----------------
MODEL_PATH = "/content/drive/MyDrive/plastic_classifier_model.h5"
BASE_PATH = "/content/drive/MyDrive/restructure_material"
SEED_FOLDER = "/content/drive/MyDrive/plant_seed"

# ---------------- MODEL ----------------
try:
    model = load_model(MODEL_PATH, compile=False)
    print("✅ Model loaded successfully!")
except Exception as e:
    print("⚠️ Model load failed, using dummy:", e)
    class Dummy:
        def predict(self, x): return np.array([[0.9, 0.1]])
    model = Dummy()

# ---------------- HELPERS (RESTRUCTURING - UNCHANGED) ----------------
def classify_uploaded_image(uploaded_path):
    if not uploaded_path:
        return "⚠️ Upload an image first.", None
    img = Image.open(uploaded_path).convert("RGB").resize((160,160))
    arr = np.expand_dims(np.array(img)/255., axis=0)
    pred = model.predict(arr)
    pred = np.array(pred).squeeze()
    if pred.size == 1:
        idx = 0 if pred < 0.5 else 1
        conf = float(pred)*100
    else:
        idx = int(np.argmax(pred))
        conf = float(np.max(pred))*100
    material = "bottle" if idx==0 else "container"
    return f"✅ Predicted Material: {material.upper()}", material

def load_category_images(material):
    folder = os.path.join(BASE_PATH, material.lower())
    if not os.path.exists(folder):
        return [], [], []
    files = [f for f in os.listdir(folder) if f.lower().endswith(('.jpg','.jpeg','.png'))]
    soil = [os.path.join(folder,f) for f in files if 'soil' in f.lower()]
    hydro = [os.path.join(folder,f) for f in files if 'hydro' in f.lower()]
    aero = [os.path.join(folder,f) for f in files if 'aero' in f.lower()]
    return soil, hydro, aero

def get_instructions(material):
    general_instructions = """
### 🧭 General Instructions
- ♻️ Clean the waste material thoroughly before reuse.
- 🪚 Cut or modify safely using proper tools.
- 🌱 Choose a farming method (Soil-based, Hydroponics, Aeroponics).
- 💧 Maintain correct watering or nutrient levels.
- ☀️ Provide adequate sunlight or controlled temperature.
"""

    soil_instructions = """
### 🌱 Soil-based Farming
1. Fill with soil and compost mix.
2. Add sapling and ensure drainage holes.
3. Place under sunlight and water regularly.
"""

    hydro_instructions = """
### 💧 Hydroponics
1. Fill partially with nutrient solution.
2. Support roots using cocopeat or sponge.
3. Maintain aeration and pH balance.
"""

    aero_instructions = """
### 🌬️ Aeroponics
1. Suspend roots in air and mist periodically.
2. Keep spray nozzles clean.
3. Maintain humidity and temperature.
"""

    return general_instructions, soil_instructions, hydro_instructions, aero_instructions

# ---------------- CALLBACKS (RESTRUCTURING - UNCHANGED) ----------------
def on_upload(img_path):
    status, material = classify_uploaded_image(img_path)
    general, soil_inst, hydro_inst, aero_inst = get_instructions(material)

    if material is None:
        return status, general, "", "", "", [], [], [], "No selection yet."

    soil, hydro, aero = load_category_images(material)

    def to_gallery(paths):
        return [(p, os.path.basename(p)) for p in paths if os.path.exists(p)]

    return status, general, soil_inst, hydro_inst, aero_inst, to_gallery(soil), to_gallery(hydro), to_gallery(aero), "No selection yet."


def on_gallery_select(evt: gr.SelectData, gallery_data):
    """Handle gallery selection with proper error handling"""
    try:
        selected_index = evt.index
        if gallery_data is None or len(gallery_data) == 0:
            return "⚠️ No images in gallery"

        if selected_index >= len(gallery_data):
            return "⚠️ Invalid selection"

        selected_item = gallery_data[selected_index]
        selected_path = selected_item[0]
        selected_name = os.path.basename(selected_path)
        return selected_name
    except Exception as e:
        return f"⚠️ Error: {str(e)}"


def confirm_selected(selected_name):
    """Confirm the selection when button is clicked"""
    if not selected_name or "⚠️" in selected_name or "No selection" in selected_name:
        return "⚠️ No design selected. Click a thumbnail first."
    return f"✅ You selected: **{selected_name}**"


# ============================================================================
# -------------------- SEEDLING FUNCTIONALITY (NEW) --------------------------
# ============================================================================

# Mapping plant names to actual image filenames
SEED_IMAGE_MAP = {
    "Tomato": "tomato.jpg",
    "Brinjal": "brinjal.jpg",
    "Ladies Finger": "ladiesfinger.jpeg",
    "Ridge Gaurd": "Ridge gaurd.jpg",
    "Snake Gaurd": "snakegourd.jpg",
    "Bootle Gaurd": "bottlegourd.jpg",
    "Beans": "beans.jpg",
    "Cluster Beans": "clusterbeans.jpg",
    "Corriander": "corriander.jpeg",
    "Fenugreek": "fenugreek.jpeg",
    "Cucumber": "cucumber.jpg",
    "Spinach": "spinach.jpeg",
    "Mint": "mint.jpeg",
    "Basil": "basil.jpeg",
    "Chilli": "chilli.jpeg",
    "Lemon Grass": "lemongrass.jpeg",
    "Aloe Vera": "aloevera.jpg",
    "Lettuce": "lettuce.jpeg",
    "Celery": "celery.jpeg",
    "Water Spinach": "waterspinach.jpeg",
    "Thyme": "thyme.jpeg",
    "Microgreens": "microgreen.jpeg",
    "Tomato Cherry": "tomatocherry.jpeg"
}

# Plant database with compatibility
PLANTS = [
    {"name": "Tomato", "soil": True, "hydro": True, "aero": True},
    {"name": "Brinjal", "soil": True, "hydro": True, "aero": False},
    {"name": "Ladies Finger", "soil": True, "hydro": True, "aero": False},
    {"name": "Ridge Gaurd", "soil": True, "hydro": False, "aero": False},
    {"name": "Snake Gaurd", "soil": True, "hydro": False, "aero": False},
    {"name": "Bootle Gaurd", "soil": False, "hydro": True, "aero": False},
    {"name": "Beans", "soil": True, "hydro": True, "aero": False},
    {"name": "Cluster Beans", "soil": True, "hydro": False, "aero": False},
    {"name": "Corriander", "soil": True, "hydro": True, "aero": True},
    {"name": "Fenugreek", "soil": True, "hydro": True, "aero": True},
    {"name": "Cucumber", "soil": True, "hydro": False, "aero": True},
    {"name": "Spinach", "soil": False, "hydro": False, "aero": True},
    {"name": "Mint", "soil": False, "hydro": False, "aero": True},
    {"name": "Basil", "soil": True, "hydro": True, "aero": True},
    {"name": "Chilli", "soil": True, "hydro": True, "aero": False},
    {"name": "Lemon Grass", "soil": True, "hydro": False, "aero": False},
    {"name": "Aloe Vera", "soil": True, "hydro": False, "aero": False},
    {"name": "Lettuce", "soil": False, "hydro": True, "aero": True},
    {"name": "Celery", "soil": False, "hydro": True, "aero": False},
    {"name": "Water Spinach", "soil": False, "hydro": True, "aero": True},
    {"name": "Thyme", "soil": False, "hydro": False, "aero": True},
    {"name": "Microgreens", "soil": False, "hydro": False, "aero": True}
]

# Seed prices
SEED_PRICES = {
    "Tomato": "₹20-40 per packet", "Brinjal": "₹15-30 per packet",
    "Ladies Finger": "₹20-35 per packet", "Ridge Gaurd": "₹25-40 per packet",
    "Snake Gaurd": "₹25-40 per packet", "Bootle Gaurd": "₹20-35 per packet",
    "Beans": "₹30-50 per packet", "Cluster Beans": "₹20-35 per packet",
    "Corriander": "₹10-20 per packet", "Fenugreek": "₹10-20 per packet",
    "Cucumber": "₹20-40 per packet", "Spinach": "₹15-30 per packet",
    "Mint": "₹15-25 per packet", "Basil": "₹20-35 per packet",
    "Chilli": "₹25-45 per packet", "Lemon Grass": "₹30-50 per packet",
    "Aloe Vera": "₹40-80 per plant", "Lettuce": "₹20-40 per packet",
    "Celery": "₹30-50 per packet", "Water Spinach": "₹15-30 per packet",
    "Thyme": "₹25-45 per packet", "Microgreens": "₹50-100 per packet"
}

# Growing instructions
GROWING_TIPS = {
    "Tomato": "Sow seeds 1cm deep in moist soil; Transplant after 3-4 weeks when seedlings are 6 inches tall",
    "Brinjal": "Start seeds indoors in seedling trays; Transfer to containers after 5-6 weeks",
    "Ladies Finger": "Direct sow seeds 2cm deep in warm soil; Space plants 12 inches apart",
    "Ridge Gaurd": "Soak seeds overnight before planting; Provide vertical support for climbing",
    "Snake Gaurd": "Plant seeds in nutrient-rich compost; Needs trellis or support structure",
    "Bootle Gaurd": "Start in small pots and transplant; Requires strong support for heavy fruits",
    "Beans": "Sow directly 3-4cm deep in well-drained soil; Water regularly but avoid overwatering",
    "Cluster Beans": "Plant seeds 2cm deep in rows; Thrives in warm weather with minimal care",
    "Corriander": "Scatter seeds lightly on soil surface; Keep moist until germination in 7-10 days",
    "Fenugreek": "Soak seeds for 12 hours before sowing; Harvest leaves in 3-4 weeks",
    "Cucumber": "Plant 2-3 seeds per hole 2cm deep; Keep soil consistently moist",
    "Spinach": "Sow seeds thinly in shallow furrows; Harvest outer leaves regularly",
    "Mint": "Use stem cuttings or seeds in moist soil; Grows rapidly and spreads quickly",
    "Basil": "Start seeds indoors or in warm spot; Pinch off flowers to promote leaf growth",
    "Chilli": "Start indoors 8-10 weeks before transplanting; Needs warm temperatures to fruit",
    "Lemon Grass": "Plant stalks from grocery store in water; Transfer to soil after roots develop",
    "Aloe Vera": "Use offsets or pups from mother plant; Needs minimal watering and well-draining soil",
    "Lettuce": "Sow seeds on surface, press lightly; Harvest leaves in 4-6 weeks",
    "Celery": "Start seeds indoors in trays; Transplant after 10-12 weeks",
    "Water Spinach": "Plant stem cuttings in water or moist soil; Grows quickly in warm conditions",
    "Thyme": "Surface sow seeds or use cuttings; Requires well-draining soil and sunlight",
    "Microgreens": "Densely scatter seeds on moist growing medium; Harvest in 7-14 days"
}

def extract_farming_type(filename):
    """Extract farming type from filename"""
    if not filename or "No selection" in filename or "⚠️" in filename:
        return None
    filename_lower = filename.lower()
    if 'soil' in filename_lower:
        return 'soil'
    elif 'hydro' in filename_lower:
        return 'hydro'
    elif 'aero' in filename_lower:
        return 'aero'
    return None

def get_suitable_plants(farming_type):
    """Get plants suitable for the farming type"""
    suitable = []
    for plant in PLANTS:
        if plant.get(farming_type, False):
            suitable.append(plant["name"])
    random.shuffle(suitable)
    return suitable[:5]

def get_seed_image(plant_name):
    """Get seed image path using the mapping"""
    filename = SEED_IMAGE_MAP.get(plant_name)
    if filename:
        img_path = os.path.join(SEED_FOLDER, filename)
        if os.path.exists(img_path):
            return img_path
    return None

def generate_seedling_info(selected_name):
    """Generate seedling recommendations when 'Show Seedlings' is clicked"""
    farming_type = extract_farming_type(selected_name)

    if not farming_type:
        return gr.update(visible=False), ""

    plants = get_suitable_plants(farming_type)

    if not plants:
        return gr.update(visible=True), "<div><h3>🌱 Seedling Recommendations</h3><p>No suitable plants found for this farming type.</p></div>"

    recommendations_html = f"""
    <div style="font-family: Arial, sans-serif;">
        <h1>🌱 Seedling Recommendations</h1>
        <h3 style="color: #2e7d32;">Based on your selected {farming_type.upper()}-based farming design</h3>
        <hr style="border: 1px solid #ddd; margin: 20px 0;">
    """

    for plant_name in plants:
        img_path = get_seed_image(plant_name)
        price = SEED_PRICES.get(plant_name, "₹20-50 per packet")
        tips = GROWING_TIPS.get(plant_name, "Sow seeds in moist soil; Keep in warm, sunny location")

        recommendations_html += f"""
        <div style="margin-bottom: 30px; padding: 15px; background: #f5f5f5; border-radius: 10px;">
            <h2 style="color: #2e7d32;">🌿 {plant_name}</h2>
        """

        if img_path and os.path.exists(img_path):
            # Convert to proper file path
            file_path = img_path.replace("\\", "/")
            recommendations_html += f"""
            <div style="margin: 15px 0;">
                <img src="file={file_path}" style="width: 200px; border-radius: 10px; box-shadow: 0 2px 8px rgba(0,0,0,0.1);">
            </div>
            """
        else:
            recommendations_html += f"""
            <p style="color: #999; font-style: italic; margin: 15px 0;">📷 Image not available</p>
            """

        recommendations_html += f"""
            <div style="margin-top: 15px;">
                <p><strong>Growing Instructions:</strong></p>
                <ul style="line-height: 1.8;">
                    <li>🏡 {tips}</li>
                    <li>💰 Market Price: {price}</li>
                </ul>
            </div>
        </div>
        <hr style="border: 0.5px solid #ddd; margin: 20px 0;">
        """

    recommendations_html += "</div>"

    return gr.update(visible=True), recommendations_html


# In the UI section, change the seedling_box from Markdown to HTML:
# Replace this line:
# seedling_box = gr.Markdown("")
# With:
# ============================================================================
# -------------------- UI LAYOUT WITH TABS -----------------------------------
# ============================================================================

with gr.Blocks(theme=gr.themes.Soft(primary_hue="green")) as app:
    gr.Markdown(
        """
        # ♻️ Smart Waste Restructure System
        Transform plastic waste into farming containers and discover the perfect plants to grow!
        """
    )

    with gr.Tabs():
        # ==================== TAB 1: RESTRUCTURING ====================
        with gr.Tab("♻️ Restructuring"):
            gr.Markdown(
                """
                Upload an image of a **plastic waste** item to classify it (bottle/container).
                Then explore eco-friendly **reuse ideas** under Soil, Hydro, and Aeroponics 🌱
                """
            )

            with gr.Row():
                upload = gr.Image(label="📤 Upload Material", type="filepath", height=160)
                status = gr.Textbox(label="Prediction", interactive=False)

            general_box = gr.Markdown("Upload an image to get started.")

            # Soil-based Farming Section
            gr.Markdown("---")
            soil_inst_box = gr.Markdown("")
            soil_gallery = gr.Gallery(columns=6, height=150, label="Soil-based Farming", object_fit="cover", container=False, show_label=True)

            # Hydroponics Section
            gr.Markdown("---")
            hydro_inst_box = gr.Markdown("")
            hydro_gallery = gr.Gallery(columns=6, height=150, label="Hydroponics", object_fit="cover", container=False, show_label=True)

            # Aeroponics Section
            gr.Markdown("---")
            aero_inst_box = gr.Markdown("")
            aero_gallery = gr.Gallery(columns=6, height=150, label="Aeroponics", object_fit="cover", container=False, show_label=True)
            gr.Markdown("---")

            with gr.Row():
                choose_btn = gr.Button("Select this design")
                selected_text = gr.Markdown("No selection yet.")

            # Wiring for restructuring
            upload.change(
                on_upload,
                inputs=upload,
                outputs=[status, general_box, soil_inst_box, hydro_inst_box, aero_inst_box,
                         soil_gallery, hydro_gallery, aero_gallery, selected_text]
            )

            soil_gallery.select(
                on_gallery_select,
                inputs=[soil_gallery],
                outputs=[selected_text]
            )

            hydro_gallery.select(
                on_gallery_select,
                inputs=[hydro_gallery],
                outputs=[selected_text]
            )

            aero_gallery.select(
                on_gallery_select,
                inputs=[aero_gallery],
                outputs=[selected_text]
            )

            choose_btn.click(confirm_selected, inputs=selected_text, outputs=selected_text)

        # ==================== TAB 2: SEEDLINGS ====================
        with gr.Tab("🌱 Seedlings"):
            gr.Markdown(
                """
                ## Get Personalized Plant Recommendations
                After selecting a restructuring design in the previous tab, come here to discover
                which plants are perfect for your chosen farming method!
                """
            )

            gr.Markdown("---")

            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("""
                    ### How to use:
                    1. Select a design in the **Restructuring** tab
                    2. Come back to this tab
                    3. Click "Show Seedlings" below
                    4. Get 5 recommended plants with images and growing tips!
                    """)
                    seedling_btn = gr.Button("🌿 Show Seedlings", variant="primary", size="lg")

                with gr.Column(scale=2):
                    seedling_section = gr.Column(visible=False)
                    with seedling_section:
                        seedling_box = gr.HTML("")

            # Wiring for seedling functionality
            seedling_btn.click(
                generate_seedling_info,
                inputs=selected_text,
                outputs=[seedling_section, seedling_box]
            )

app.launch(share=True)

In [ ]:
!pip install gradio --quiet
import gradio as gr
import os
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model
import random

# ---------------- CONFIG ----------------
MODEL_PATH = "/content/drive/MyDrive/plastic_classifier_model.h5"
BASE_PATH = "/content/drive/MyDrive/restructure_material"
SEED_FOLDER = "/content/drive/MyDrive/plant_seed"

# ---------------- MODEL ----------------
try:
    model = load_model(MODEL_PATH, compile=False)
    print("✅ Model loaded successfully!")
except Exception as e:
    print("⚠️ Model load failed, using dummy:", e)
    class Dummy:
        def predict(self, x): return np.array([[0.9, 0.1]])
    model = Dummy()

# ---------------- HELPERS (RESTRUCTURING - UNCHANGED) ----------------
def classify_uploaded_image(uploaded_path):
    if not uploaded_path:
        return "⚠️ Upload an image first.", None
    img = Image.open(uploaded_path).convert("RGB").resize((160,160))
    arr = np.expand_dims(np.array(img)/255., axis=0)
    pred = model.predict(arr)
    pred = np.array(pred).squeeze()
    if pred.size == 1:
        idx = 0 if pred < 0.5 else 1
        conf = float(pred)*100
    else:
        idx = int(np.argmax(pred))
        conf = float(np.max(pred))*100
    material = "bottle" if idx==0 else "container"
    return f"✅ Predicted Material: {material.upper()}", material

def load_category_images(material):
    folder = os.path.join(BASE_PATH, material.lower())
    if not os.path.exists(folder):
        return [], [], []
    files = [f for f in os.listdir(folder) if f.lower().endswith(('.jpg','.jpeg','.png'))]
    soil = [os.path.join(folder,f) for f in files if 'soil' in f.lower()]
    hydro = [os.path.join(folder,f) for f in files if 'hydro' in f.lower()]
    aero = [os.path.join(folder,f) for f in files if 'aero' in f.lower()]
    return soil, hydro, aero

def get_instructions(material):
    general_instructions = """
### 🧭 General Instructions
- ♻️ Clean the waste material thoroughly before reuse.
- 🪚 Cut or modify safely using proper tools.
- 🌱 Choose a farming method (Soil-based, Hydroponics, Aeroponics).
- 💧 Maintain correct watering or nutrient levels.
- ☀️ Provide adequate sunlight or controlled temperature.
"""

    soil_instructions = """
### 🌱 Soil-based Farming
1. Fill with soil and compost mix.
2. Add sapling and ensure drainage holes.
3. Place under sunlight and water regularly.
"""

    hydro_instructions = """
### 💧 Hydroponics
1. Fill partially with nutrient solution.
2. Support roots using cocopeat or sponge.
3. Maintain aeration and pH balance.
"""

    aero_instructions = """
### 🌬️ Aeroponics
1. Suspend roots in air and mist periodically.
2. Keep spray nozzles clean.
3. Maintain humidity and temperature.
"""

    return general_instructions, soil_instructions, hydro_instructions, aero_instructions

# ---------------- CALLBACKS (RESTRUCTURING - UNCHANGED) ----------------
def on_upload(img_path):
    status, material = classify_uploaded_image(img_path)
    general, soil_inst, hydro_inst, aero_inst = get_instructions(material)

    if material is None:
        return status, general, "", "", "", [], [], [], "No selection yet."

    soil, hydro, aero = load_category_images(material)

    def to_gallery(paths):
        return [(p, os.path.basename(p)) for p in paths if os.path.exists(p)]

    return status, general, soil_inst, hydro_inst, aero_inst, to_gallery(soil), to_gallery(hydro), to_gallery(aero), "No selection yet."


def on_gallery_select(evt: gr.SelectData, gallery_data):
    """Handle gallery selection with proper error handling"""
    try:
        selected_index = evt.index
        if gallery_data is None or len(gallery_data) == 0:
            return "⚠️ No images in gallery"

        if selected_index >= len(gallery_data):
            return "⚠️ Invalid selection"

        selected_item = gallery_data[selected_index]
        selected_path = selected_item[0]
        selected_name = os.path.basename(selected_path)
        return selected_name
    except Exception as e:
        return f"⚠️ Error: {str(e)}"


def confirm_selected(selected_name):
    """Confirm the selection when button is clicked"""
    if not selected_name or "⚠️" in selected_name or "No selection" in selected_name:
        return "⚠️ No design selected. Click a thumbnail first."
    return f"✅ You selected: **{selected_name}**"


# ============================================================================
# -------------------- SEEDLING FUNCTIONALITY (FIXED) ------------------------
# ============================================================================

# Mapping plant names to actual image filenames
SEED_IMAGE_MAP = {
    "Tomato": "Tomato.jpg",
    "Brinjal": "Brinjal.jpg",
    "Ladies Finger": "Ladies Finger.jpeg",
    "Ridge Gaurd": "Ridge Gaurd.jpg",
    "Snake Gaurd": "Snake Gaurd.jpg",
    "Bootle Gaurd": "Bottle Gaurd.jpg",
    "Beans": "Beans.jpg",
    "Cluster Beans": "Cluster Beans.jpg",
    "Corriander": "Corriander.jpeg",
    "Fenugreek": "Fenugreek.jpeg",
    "Cucumber": "Cucumber.jpg",
    "Spinach": "Spinach.jpeg",
    "Mint": "Mint.jpeg",
    "Basil": "Basil.jpeg",
    "Chilli": "Chilli.jpeg",
    "Lemon Grass": "Lemon Grass.jpeg",
    "Aloe Vera": "Aloe Vera.jpg",
    "Lettuce": "Lettuce.jpeg",
    "Celery": "Celery.jpeg",
    "Water Spinach": "Water Spinach.jpeg",
    "Thyme": "Thyme.jpeg",
    "Microgreens": "Microgreens.jpeg",
    "Tomato Cherry": "tomatocherry.jpeg"
}

# Plant database with compatibility
PLANTS = [
    {"name": "Tomato", "soil": True, "hydro": True, "aero": True},
    {"name": "Brinjal", "soil": True, "hydro": True, "aero": False},
    {"name": "Ladies Finger", "soil": True, "hydro": True, "aero": False},
    {"name": "Ridge Gaurd", "soil": True, "hydro": False, "aero": False},
    {"name": "Snake Gaurd", "soil": True, "hydro": False, "aero": False},
    {"name": "Bootle Gaurd", "soil": False, "hydro": True, "aero": False},
    {"name": "Beans", "soil": True, "hydro": True, "aero": False},
    {"name": "Cluster Beans", "soil": True, "hydro": False, "aero": False},
    {"name": "Corriander", "soil": True, "hydro": True, "aero": True},
    {"name": "Fenugreek", "soil": True, "hydro": True, "aero": True},
    {"name": "Cucumber", "soil": True, "hydro": False, "aero": True},
    {"name": "Spinach", "soil": False, "hydro": False, "aero": True},
    {"name": "Mint", "soil": False, "hydro": False, "aero": True},
    {"name": "Basil", "soil": True, "hydro": True, "aero": True},
    {"name": "Chilli", "soil": True, "hydro": True, "aero": False},
    {"name": "Lemon Grass", "soil": True, "hydro": False, "aero": False},
    {"name": "Aloe Vera", "soil": True, "hydro": False, "aero": False},
    {"name": "Lettuce", "soil": False, "hydro": True, "aero": True},
    {"name": "Celery", "soil": False, "hydro": True, "aero": False},
    {"name": "Water Spinach", "soil": False, "hydro": True, "aero": True},
    {"name": "Thyme", "soil": False, "hydro": False, "aero": True},
    {"name": "Microgreens", "soil": False, "hydro": False, "aero": True}
]

# Seed prices
SEED_PRICES = {
    "Tomato": "₹20-40 per packet", "Brinjal": "₹15-30 per packet",
    "Ladies Finger": "₹20-35 per packet", "Ridge Gaurd": "₹25-40 per packet",
    "Snake Gaurd": "₹25-40 per packet", "Bootle Gaurd": "₹20-35 per packet",
    "Beans": "₹30-50 per packet", "Cluster Beans": "₹20-35 per packet",
    "Corriander": "₹10-20 per packet", "Fenugreek": "₹10-20 per packet",
    "Cucumber": "₹20-40 per packet", "Spinach": "₹15-30 per packet",
    "Mint": "₹15-25 per packet", "Basil": "₹20-35 per packet",
    "Chilli": "₹25-45 per packet", "Lemon Grass": "₹30-50 per packet",
    "Aloe Vera": "₹40-80 per plant", "Lettuce": "₹20-40 per packet",
    "Celery": "₹30-50 per packet", "Water Spinach": "₹15-30 per packet",
    "Thyme": "₹25-45 per packet", "Microgreens": "₹50-100 per packet"
}

# Growing instructions
GROWING_TIPS = {
    "Tomato": "Sow seeds 1cm deep in moist soil; Transplant after 3-4 weeks when seedlings are 6 inches tall",
    "Brinjal": "Start seeds indoors in seedling trays; Transfer to containers after 5-6 weeks",
    "Ladies Finger": "Direct sow seeds 2cm deep in warm soil; Space plants 12 inches apart",
    "Ridge Gaurd": "Soak seeds overnight before planting; Provide vertical support for climbing",
    "Snake Gaurd": "Plant seeds in nutrient-rich compost; Needs trellis or support structure",
    "Bootle Gaurd": "Start in small pots and transplant; Requires strong support for heavy fruits",
    "Beans": "Sow directly 3-4cm deep in well-drained soil; Water regularly but avoid overwatering",
    "Cluster Beans": "Plant seeds 2cm deep in rows; Thrives in warm weather with minimal care",
    "Corriander": "Scatter seeds lightly on soil surface; Keep moist until germination in 7-10 days",
    "Fenugreek": "Soak seeds for 12 hours before sowing; Harvest leaves in 3-4 weeks",
    "Cucumber": "Plant 2-3 seeds per hole 2cm deep; Keep soil consistently moist",
    "Spinach": "Sow seeds thinly in shallow furrows; Harvest outer leaves regularly",
    "Mint": "Use stem cuttings or seeds in moist soil; Grows rapidly and spreads quickly",
    "Basil": "Start seeds indoors or in warm spot; Pinch off flowers to promote leaf growth",
    "Chilli": "Start indoors 8-10 weeks before transplanting; Needs warm temperatures to fruit",
    "Lemon Grass": "Plant stalks from grocery store in water; Transfer to soil after roots develop",
    "Aloe Vera": "Use offsets or pups from mother plant; Needs minimal watering and well-draining soil",
    "Lettuce": "Sow seeds on surface, press lightly; Harvest leaves in 4-6 weeks",
    "Celery": "Start seeds indoors in trays; Transplant after 10-12 weeks",
    "Water Spinach": "Plant stem cuttings in water or moist soil; Grows quickly in warm conditions",
    "Thyme": "Surface sow seeds or use cuttings; Requires well-draining soil and sunlight",
    "Microgreens": "Densely scatter seeds on moist growing medium; Harvest in 7-14 days"
}

def extract_farming_type(filename):
    """Extract farming type from filename"""
    if not filename or "No selection" in filename or "⚠️" in filename:
        return None
    filename_lower = filename.lower()
    if 'soil' in filename_lower:
        return 'soil'
    elif 'hydro' in filename_lower:
        return 'hydro'
    elif 'aero' in filename_lower:
        return 'aero'
    return None

def get_suitable_plants(farming_type):
    """Get plants suitable for the farming type"""
    suitable = []
    for plant in PLANTS:
        if plant.get(farming_type, False):
            suitable.append(plant["name"])
    random.shuffle(suitable)
    return suitable[:5]

def get_seed_image(plant_name):
    """Get seed image path - WITH DEBUGGING"""
    filename = SEED_IMAGE_MAP.get(plant_name)

    print(f"\n{'='*60}")
    print(f"🔍 Searching for plant: {plant_name}")
    print(f"📝 Expected filename: {filename}")
    print(f"📁 Looking in folder: {SEED_FOLDER}")
    print(f"📁 Folder exists: {os.path.exists(SEED_FOLDER)}")

    if not filename:
        print(f"❌ No filename mapping found for {plant_name}")
        return None

    img_path = os.path.join(SEED_FOLDER, filename)
    print(f"🔗 Full path: {img_path}")
    print(f"✓ File exists: {os.path.exists(img_path)}")

    if os.path.exists(img_path):
        print(f"✅ SUCCESS: Found image at {img_path}")
        return img_path
    else:
        print(f"❌ File not found at exact path")
        # List all files in the folder
        if os.path.exists(SEED_FOLDER):
            all_files = os.listdir(SEED_FOLDER)
            print(f"📂 Files in seed folder ({len(all_files)} total):")
            for f in sorted(all_files):
                print(f"   - {f}")

            # Try case-insensitive search
            print(f"\n🔄 Trying case-insensitive match...")
            for actual_file in all_files:
                if actual_file.lower() == filename.lower():
                    found_path = os.path.join(SEED_FOLDER, actual_file)
                    print(f"✅ FOUND with case mismatch: {actual_file}")
                    print(f"✅ Using path: {found_path}")
                    return found_path

            print(f"❌ No case-insensitive match found either")
        else:
            print(f"❌ Seed folder doesn't exist!")

        return None


def generate_seedling_info(selected_name):
    """Generate seedling recommendations - USING GALLERY LIKE RESTRUCTURING"""
    print(f"\n{'='*60}")
    print(f"🌱 GENERATE SEEDLING INFO CALLED")
    print(f"📝 Selected name: {selected_name}")

    farming_type = extract_farming_type(selected_name)
    print(f"🌾 Extracted farming type: {farming_type}")

    if not farming_type:
        print(f"❌ No farming type extracted")
        return gr.update(visible=False), "", []

    plants = get_suitable_plants(farming_type)
    print(f"🌿 Suitable plants: {plants}")

    if not plants:
        return gr.update(visible=True), "### 🌱 Seedling Recommendations\n\nNo suitable plants found for this farming type.", []

    recommendations_text = f"""
# 🌱 Seedling Recommendations
### Based on your selected {farming_type.upper()}-based farming design

---
"""

    # Prepare gallery data - SAME FORMAT AS RESTRUCTURING
    gallery_data = []

    for plant_name in plants:
        img_path = get_seed_image(plant_name)
        price = SEED_PRICES.get(plant_name, "₹20-50 per packet")
        tips = GROWING_TIPS.get(plant_name, "Sow seeds in moist soil; Keep in warm, sunny location")

        recommendations_text += f"""
## 🌿 {plant_name}

**Growing Instructions:**
- 🏡 {tips}
- 💰 Market Price: {price}

---
"""

        # Add to gallery - EXACTLY LIKE load_category_images does it
        if img_path and os.path.exists(img_path):
            gallery_data.append((img_path, plant_name))
            print(f"✅ Added to gallery: {plant_name} -> {img_path}")
        else:
            print(f"❌ NOT added to gallery: {plant_name} (path invalid)")

    print(f"\n📊 Gallery data summary:")
    print(f"   Total items: {len(gallery_data)}")
    for item in gallery_data:
        print(f"   - {item[1]}: {item[0]}")

    print(f"{'='*60}\n")

    return gr.update(visible=True), recommendations_text, gallery_data


# ============================================================================
# -------------------- UI LAYOUT WITH TABS -----------------------------------
# ============================================================================

with gr.Blocks(theme=gr.themes.Soft(primary_hue="green")) as app:
    gr.Markdown(
        """
        # ♻️ Smart Waste Restructure System
        Transform plastic waste into farming containers and discover the perfect plants to grow!
        """
    )

    with gr.Tabs():
        # ==================== TAB 1: RESTRUCTURING ====================
        with gr.Tab("♻️ Restructuring"):
            gr.Markdown(
                """
                Upload an image of a **plastic waste** item to classify it (bottle/container).
                Then explore eco-friendly **reuse ideas** under Soil, Hydro, and Aeroponics 🌱
                """
            )

            with gr.Row():
                upload = gr.Image(label="📤 Upload Material", type="filepath", height=160)
                status = gr.Textbox(label="Prediction", interactive=False)

            general_box = gr.Markdown("Upload an image to get started.")

            # Soil-based Farming Section
            gr.Markdown("---")
            soil_inst_box = gr.Markdown("")
            soil_gallery = gr.Gallery(columns=6, height=150, label="Soil-based Farming", object_fit="cover", container=False, show_label=True)

            # Hydroponics Section
            gr.Markdown("---")
            hydro_inst_box = gr.Markdown("")
            hydro_gallery = gr.Gallery(columns=6, height=150, label="Hydroponics", object_fit="cover", container=False, show_label=True)

            # Aeroponics Section
            gr.Markdown("---")
            aero_inst_box = gr.Markdown("")
            aero_gallery = gr.Gallery(columns=6, height=150, label="Aeroponics", object_fit="cover", container=False, show_label=True)
            gr.Markdown("---")

            with gr.Row():
                choose_btn = gr.Button("Select this design")
                selected_text = gr.Markdown("No selection yet.")

            # Wiring for restructuring
            upload.change(
                on_upload,
                inputs=upload,
                outputs=[status, general_box, soil_inst_box, hydro_inst_box, aero_inst_box,
                         soil_gallery, hydro_gallery, aero_gallery, selected_text]
            )

            soil_gallery.select(
                on_gallery_select,
                inputs=[soil_gallery],
                outputs=[selected_text]
            )

            hydro_gallery.select(
                on_gallery_select,
                inputs=[hydro_gallery],
                outputs=[selected_text]
            )

            aero_gallery.select(
                on_gallery_select,
                inputs=[aero_gallery],
                outputs=[selected_text]
            )

            choose_btn.click(confirm_selected, inputs=selected_text, outputs=selected_text)

        # ==================== TAB 2: SEEDLINGS ====================
        with gr.Tab("🌱 Seedlings"):
            gr.Markdown(
                """
                ## Get Personalized Plant Recommendations
                After selecting a restructuring design in the previous tab, come here to discover
                which plants are perfect for your chosen farming method!
                """
            )

            gr.Markdown("---")

            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("""
                    ### How to use:
                    1. Select a design in the **Restructuring** tab
                    2. Come back to this tab
                    3. Click "Show Seedlings" below
                    4. Get 5 recommended plants with images and growing tips!
                    """)
                    seedling_btn = gr.Button("🌿 Show Seedlings", variant="primary", size="lg")

            seedling_section = gr.Column(visible=False)
            with seedling_section:
                seedling_info_box = gr.Markdown("")
                # GALLERY - EXACTLY LIKE RESTRUCTURING GALLERIES
                seedling_gallery = gr.Gallery(
                    columns=5,
                    height=200,
                    label="🌱 Recommended Seeds",
                    object_fit="cover",  # Changed from "contain" to "cover" like restructuring
                    container=False,     # Changed to False like restructuring
                    show_label=True
                )

            # Wiring for seedling functionality
            seedling_btn.click(
                generate_seedling_info,
                inputs=selected_text,
                outputs=[seedling_section, seedling_info_box, seedling_gallery]
            )

app.launch(share=True)

In [ ]:
!pip install gradio --quiet
import gradio as gr
import os
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model
import random

# ---------------- CONFIG ----------------
MODEL_PATH = "/content/drive/MyDrive/plastic_classifier_model.h5"
BASE_PATH = "/content/drive/MyDrive/restructure_material"
SEED_FOLDER = "/content/drive/MyDrive/plant_seeds"

# ---------------- MODEL ----------------
try:
    model = load_model(MODEL_PATH, compile=False)
    print("✅ Model loaded successfully!")
except Exception as e:
    print("⚠️ Model load failed, using dummy:", e)
    class Dummy:
        def predict(self, x): return np.array([[0.9, 0.1]])
    model = Dummy()

# ---------------- HELPERS (RESTRUCTURING - UNCHANGED) ----------------
def classify_uploaded_image(uploaded_path):
    if not uploaded_path:
        return "⚠️ Upload an image first.", None
    img = Image.open(uploaded_path).convert("RGB").resize((160,160))
    arr = np.expand_dims(np.array(img)/255., axis=0)
    pred = model.predict(arr)
    pred = np.array(pred).squeeze()
    if pred.size == 1:
        idx = 0 if pred < 0.5 else 1
        conf = float(pred)*100
    else:
        idx = int(np.argmax(pred))
        conf = float(np.max(pred))*100
    material = "bottle" if idx==0 else "container"
    return f"✅ Predicted Material: {material.upper()}", material

def load_category_images(material):
    folder = os.path.join(BASE_PATH, material.lower())
    if not os.path.exists(folder):
        return [], [], []
    files = [f for f in os.listdir(folder) if f.lower().endswith(('.jpg','.jpeg','.png'))]
    soil = [os.path.join(folder,f) for f in files if 'soil' in f.lower()]
    hydro = [os.path.join(folder,f) for f in files if 'hydro' in f.lower()]
    aero = [os.path.join(folder,f) for f in files if 'aero' in f.lower()]
    return soil, hydro, aero

def get_instructions(material):
    general_instructions = """
### 🧭 General Instructions
- ♻️ Clean the waste material thoroughly before reuse.
- 🪚 Cut or modify safely using proper tools.
- 🌱 Choose a farming method (Soil-based, Hydroponics, Aeroponics).
- 💧 Maintain correct watering or nutrient levels.
- ☀️ Provide adequate sunlight or controlled temperature.
"""

    soil_instructions = """
### 🌱 Soil-based Farming
1. Fill with soil and compost mix.
2. Add sapling and ensure drainage holes.
3. Place under sunlight and water regularly.
"""

    hydro_instructions = """
### 💧 Hydroponics
1. Fill partially with nutrient solution.
2. Support roots using cocopeat or sponge.
3. Maintain aeration and pH balance.
"""

    aero_instructions = """
### 🌬️ Aeroponics
1. Suspend roots in air and mist periodically.
2. Keep spray nozzles clean.
3. Maintain humidity and temperature.
"""

    return general_instructions, soil_instructions, hydro_instructions, aero_instructions

# ---------------- CALLBACKS (RESTRUCTURING - UNCHANGED) ----------------
def on_upload(img_path):
    status, material = classify_uploaded_image(img_path)
    general, soil_inst, hydro_inst, aero_inst = get_instructions(material)

    if material is None:
        return status, general, "", "", "", [], [], [], "No selection yet."

    soil, hydro, aero = load_category_images(material)

    def to_gallery(paths):
        return [(p, os.path.basename(p)) for p in paths if os.path.exists(p)]

    return status, general, soil_inst, hydro_inst, aero_inst, to_gallery(soil), to_gallery(hydro), to_gallery(aero), "No selection yet."


def on_gallery_select(evt: gr.SelectData, gallery_data):
    """Handle gallery selection with proper error handling"""
    try:
        selected_index = evt.index
        if gallery_data is None or len(gallery_data) == 0:
            return "⚠️ No images in gallery"

        if selected_index >= len(gallery_data):
            return "⚠️ Invalid selection"

        selected_item = gallery_data[selected_index]
        selected_path = selected_item[0]
        selected_name = os.path.basename(selected_path)
        return selected_name
    except Exception as e:
        return f"⚠️ Error: {str(e)}"


def confirm_selected(selected_name):
    """Confirm the selection when button is clicked"""
    if not selected_name or "⚠️" in selected_name or "No selection" in selected_name:
        return "⚠️ No design selected. Click a thumbnail first."
    return f"✅ You selected: **{selected_name}**"


# ============================================================================
# -------------------- SEEDLING FUNCTIONALITY --------------------------------
# ============================================================================

# Plant database with compatibility (matching your CSV exactly)
PLANTS = [
    {"name": "Tomato", "soil": True, "hydro": True, "aero": True},
    {"name": "Brinjal", "soil": True, "hydro": True, "aero": False},
    {"name": "Ladies Finger", "soil": True, "hydro": True, "aero": False},
    {"name": "Ridge gaurd", "soil": True, "hydro": False, "aero": False},
    {"name": "Snake gaurd", "soil": True, "hydro": False, "aero": False},
    {"name": "Bottle gaurd", "soil": False, "hydro": True, "aero": False},
    {"name": "Beans", "soil": True, "hydro": True, "aero": False},
    {"name": "Cluster beans", "soil": True, "hydro": False, "aero": False},
    {"name": "Corriander", "soil": True, "hydro": True, "aero": True},
    {"name": "Fenugreek", "soil": True, "hydro": True, "aero": True},
    {"name": "Cucumber", "soil": True, "hydro": False, "aero": True},
    {"name": "Spinach", "soil": False, "hydro": False, "aero": True},
    {"name": "Mint", "soil": False, "hydro": False, "aero": True},
    {"name": "Basil", "soil": True, "hydro": True, "aero": True},
    {"name": "Chilli", "soil": True, "hydro": True, "aero": False},
    {"name": "Lemon grass", "soil": True, "hydro": False, "aero": False},
    {"name": "Aloe vera", "soil": True, "hydro": False, "aero": False},
    {"name": "Lettuce", "soil": False, "hydro": True, "aero": True},
    {"name": "Celery", "soil": False, "hydro": True, "aero": False},
    {"name": "Tomato Cherry", "soil": False, "hydro": False, "aero": False},
    {"name": "Water Spinach", "soil": False, "hydro": True, "aero": True},
    {"name": "Thyme", "soil": False, "hydro": False, "aero": True},
    {"name": "Microgreens", "soil": False, "hydro": False, "aero": True}
]

# Seed prices
SEED_PRICES = {
    "Tomato": "₹20-40 per packet", "Brinjal": "₹15-30 per packet",
    "Ladies Finger": "₹20-35 per packet", "Ridge gaurd": "₹25-40 per packet",
    "Snake gaurd": "₹25-40 per packet", "Bottle gaurd": "₹20-35 per packet",
    "Beans": "₹30-50 per packet", "Cluster beans": "₹20-35 per packet",
    "Corriander": "₹10-20 per packet", "Fenugreek": "₹10-20 per packet",
    "Cucumber": "₹20-40 per packet", "Spinach": "₹15-30 per packet",
    "Mint": "₹15-25 per packet", "Basil": "₹20-35 per packet",
    "Chilli": "₹25-45 per packet", "Lemon grass": "₹30-50 per packet",
    "Aloe vera": "₹40-80 per plant", "Lettuce": "₹20-40 per packet",
    "Celery": "₹30-50 per packet", "Water Spinach": "₹15-30 per packet",
    "Thyme": "₹25-45 per packet", "Microgreens": "₹50-100 per packet",
    "Tomato Cherry": "₹30-60 per packet"
}

# Growing instructions
GROWING_TIPS = {
    "Tomato": "Sow seeds 1cm deep in moist soil; Transplant after 3-4 weeks when seedlings are 6 inches tall",
    "Brinjal": "Start seeds indoors in seedling trays; Transfer to containers after 5-6 weeks",
    "Ladies Finger": "Direct sow seeds 2cm deep in warm soil; Space plants 12 inches apart",
    "Ridge gaurd": "Soak seeds overnight before planting; Provide vertical support for climbing",
    "Snake gaurd": "Plant seeds in nutrient-rich compost; Needs trellis or support structure",
    "Bottle gaurd": "Start in small pots and transplant; Requires strong support for heavy fruits",
    "Beans": "Sow directly 3-4cm deep in well-drained soil; Water regularly but avoid overwatering",
    "Cluster beans": "Plant seeds 2cm deep in rows; Thrives in warm weather with minimal care",
    "Corriander": "Scatter seeds lightly on soil surface; Keep moist until germination in 7-10 days",
    "Fenugreek": "Soak seeds for 12 hours before sowing; Harvest leaves in 3-4 weeks",
    "Cucumber": "Plant 2-3 seeds per hole 2cm deep; Keep soil consistently moist",
    "Spinach": "Sow seeds thinly in shallow furrows; Harvest outer leaves regularly",
    "Mint": "Use stem cuttings or seeds in moist soil; Grows rapidly and spreads quickly",
    "Basil": "Start seeds indoors or in warm spot; Pinch off flowers to promote leaf growth",
    "Chilli": "Start indoors 8-10 weeks before transplanting; Needs warm temperatures to fruit",
    "Lemon grass": "Plant stalks from grocery store in water; Transfer to soil after roots develop",
    "Aloe vera": "Use offsets or pups from mother plant; Needs minimal watering and well-draining soil",
    "Lettuce": "Sow seeds on surface, press lightly; Harvest leaves in 4-6 weeks",
    "Celery": "Start seeds indoors in trays; Transplant after 10-12 weeks",
    "Water Spinach": "Plant stem cuttings in water or moist soil; Grows quickly in warm conditions",
    "Thyme": "Surface sow seeds or use cuttings; Requires well-draining soil and sunlight",
    "Microgreens": "Densely scatter seeds on moist growing medium; Harvest in 7-14 days",
    "Tomato Cherry": "Sow like regular tomatoes but in smaller pots; Provide support for vining varieties"
}

def extract_farming_type(filename):
    """Extract farming type from filename"""
    if not filename or "No selection" in filename or "⚠️" in filename:
        return None
    filename_lower = filename.lower()
    if 'soil' in filename_lower:
        return 'soil'
    elif 'hydro' in filename_lower:
        return 'hydro'
    elif 'aero' in filename_lower:
        return 'aero'
    return None

def get_suitable_plants(farming_type):
    """Get plants suitable for the farming type"""
    suitable = []
    for plant in PLANTS:
        if plant.get(farming_type, False):
            suitable.append(plant["name"])
    random.shuffle(suitable)
    return suitable[:5]

def load_seed_images_from_folder():
    """Load all available seed images from folder"""
    if not os.path.exists(SEED_FOLDER):
        print(f"⚠️ Seed folder not found: {SEED_FOLDER}")
        return []

    files = [f for f in os.listdir(SEED_FOLDER) if f.lower().endswith(('.jpg', '.jpeg', '.png','.webp'))]
    full_paths = [os.path.join(SEED_FOLDER, f) for f in files]

    print(f"✅ Found {len(full_paths)} seed images in folder")
    return full_paths

def match_plant_to_image(plant_name, all_image_paths):
    """Match plant name to actual image file - case insensitive"""
    plant_lower = plant_name.lower().replace(" ", "").replace("_", "")

    for img_path in all_image_paths:
        img_name = os.path.basename(img_path).lower()
        img_base = os.path.splitext(img_name)[0].replace(" ", "").replace("_", "")

        if plant_lower in img_base or img_base in plant_lower:
            return img_path

    return None

def generate_seedling_info(selected_name):
    """Generate seedling recommendations with images below each plant"""
    print(f"\n{'='*60}")
    print(f"🌱 Generating seedling recommendations")
    print(f"📝 Selected: {selected_name}")

    farming_type = extract_farming_type(selected_name)
    print(f"🌾 Farming type: {farming_type}")

    if not farming_type:
        print(f"❌ No farming type extracted")
        # Return 5 empty updates for the plant blocks
        return (gr.update(visible=False), "", None) * 5

    all_seed_images = load_seed_images_from_folder()

    if not all_seed_images:
        return (gr.update(visible=True), "⚠️ No seed images found", None) * 5

    plants = get_suitable_plants(farming_type)
    print(f"🌿 Suitable plants: {plants}")

    if not plants:
        return (gr.update(visible=True), "No suitable plants found", None) * 5

    # Prepare outputs for 5 plant blocks
    outputs = []
    for i in range(5):
        if i < len(plants):
            plant_name = plants[i]
            img_path = match_plant_to_image(plant_name, all_seed_images)

            price = SEED_PRICES.get(plant_name, "₹20-50 per packet")
            tips = GROWING_TIPS.get(plant_name, "Sow seeds in moist soil; Keep in warm, sunny location")

            plant_text = f"""
## 🌿 {plant_name}

**Growing Instructions:**
- 🏡 {tips}
- 💰 Market Price: {price}
"""

            if img_path and os.path.exists(img_path):
                print(f"✅ Matched: {plant_name} -> {os.path.basename(img_path)}")
                outputs.extend([gr.update(visible=True), plant_text, img_path])
            else:
                print(f"❌ No image found for: {plant_name}")
                outputs.extend([gr.update(visible=True), plant_text, None])
        else:
            # Hide unused plant blocks
            outputs.extend([gr.update(visible=False), "", None])

    print(f"{'='*60}\n")
    return tuple(outputs)


# ============================================================================
# -------------------- UI LAYOUT WITH TABS -----------------------------------
# ============================================================================

with gr.Blocks(theme=gr.themes.Soft(primary_hue="green")) as app:
    gr.Markdown(
        """
        # ♻️ Smart Waste Restructure System
        Transform plastic waste into farming containers and discover the perfect plants to grow!
        """
    )

    with gr.Tabs():
        # ==================== TAB 1: RESTRUCTURING ====================
        with gr.Tab("♻️ Restructuring"):
            gr.Markdown(
                """
                Upload an image of a **plastic waste** item to classify it (bottle/container).
                Then explore eco-friendly **reuse ideas** under Soil, Hydro, and Aeroponics 🌱
                """
            )

            with gr.Row():
                upload = gr.Image(label="📤 Upload Material", type="filepath", height=160)
                status = gr.Textbox(label="Prediction", interactive=False)

            general_box = gr.Markdown("Upload an image to get started.")

            # Soil-based Farming Section
            gr.Markdown("---")
            soil_inst_box = gr.Markdown("")
            soil_gallery = gr.Gallery(columns=6, height=150, label="Soil-based Farming", object_fit="cover", container=False, show_label=True)

            # Hydroponics Section
            gr.Markdown("---")
            hydro_inst_box = gr.Markdown("")
            hydro_gallery = gr.Gallery(columns=6, height=150, label="Hydroponics", object_fit="cover", container=False, show_label=True)

            # Aeroponics Section
            gr.Markdown("---")
            aero_inst_box = gr.Markdown("")
            aero_gallery = gr.Gallery(columns=6, height=150, label="Aeroponics", object_fit="cover", container=False, show_label=True)
            gr.Markdown("---")

            with gr.Row():
                choose_btn = gr.Button("Select this design")
                selected_text = gr.Markdown("No selection yet.")

            # Wiring for restructuring
            upload.change(
                on_upload,
                inputs=upload,
                outputs=[status, general_box, soil_inst_box, hydro_inst_box, aero_inst_box,
                         soil_gallery, hydro_gallery, aero_gallery, selected_text]
            )

            soil_gallery.select(
                on_gallery_select,
                inputs=[soil_gallery],
                outputs=[selected_text]
            )

            hydro_gallery.select(
                on_gallery_select,
                inputs=[hydro_gallery],
                outputs=[selected_text]
            )

            aero_gallery.select(
                on_gallery_select,
                inputs=[aero_gallery],
                outputs=[selected_text]
            )

            choose_btn.click(confirm_selected, inputs=selected_text, outputs=selected_text)

        # ==================== TAB 2: SEEDLINGS ====================
        with gr.Tab("🌱 Seedlings"):
            gr.Markdown(
                """
                ## Get Personalized Plant Recommendations
                After selecting a restructuring design in the previous tab, come here to discover
                which plants are perfect for your chosen farming method!
                """
            )

            gr.Markdown("---")

            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("""
                    ### How to use:
                    1. Select a design in the **Restructuring** tab
                    2. Come back to this tab
                    3. Click "Show Seedlings" below
                    4. Get 5 recommended plants with images and growing tips!
                    """)
                    seedling_btn = gr.Button("🌿 Show Seedlings", variant="primary", size="lg")

            gr.Markdown("---")
            gr.Markdown("# 🌱 Recommended Plants for Your Farming Method")

            # Create 5 plant blocks (each with text and image)
            plant_blocks = []
            for i in range(5):
                with gr.Column(visible=False) as plant_col:
                    plant_text = gr.Markdown("")
                    plant_img = gr.Image(label=f"Plant {i+1}", height=250, show_label=False)
                    gr.Markdown("---")
                plant_blocks.append((plant_col, plant_text, plant_img))

            # Flatten the outputs for the function call
            all_outputs = []
            for col, text, img in plant_blocks:
                all_outputs.extend([col, text, img])

            # Wiring for seedling functionality
            seedling_btn.click(
                generate_seedling_info,
                inputs=selected_text,
                outputs=all_outputs
            )

app.launch(share=True)

✅ Model loaded successfully!


/usr/local/lib/python3.12/dist-packages/gradio/components/base.py:200: UserWarning: show_label has no effect when container is False.
  warnings.warn("show_label has no effect when container is False.")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1a9b1b2ee0c3cd269a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install gradio --quiet
import gradio as gr
import os
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model
import random

# ---------------- CONFIG ----------------
MODEL_PATH = "/content/drive/MyDrive/plastic_classifier_model.h5"
BASE_PATH = "/content/drive/MyDrive/restructure_material"
SEED_FOLDER = "/content/drive/MyDrive/plant_seeds"

# ---------------- MODEL ----------------
try:
    model = load_model(MODEL_PATH, compile=False)
    print("✅ Model loaded successfully!")
except Exception as e:
    print("⚠️ Model load failed, using dummy:", e)
    class Dummy:
        def predict(self, x): return np.array([[0.9, 0.1]])
    model = Dummy()

# ---------------- HELPERS (RESTRUCTURING) ----------------
def classify_uploaded_image(uploaded_path):
    if not uploaded_path:
        return "⚠️ Upload an image first.", None
    img = Image.open(uploaded_path).convert("RGB").resize((160,160))
    arr = np.expand_dims(np.array(img)/255., axis=0)
    pred = model.predict(arr)
    pred = np.array(pred).squeeze()
    if pred.size == 1:
        idx = 0 if pred < 0.5 else 1
        conf = float(pred)*100
    else:
        idx = int(np.argmax(pred))
        conf = float(np.max(pred))*100
    material = "bottle" if idx==0 else "container"
    return f"✅ Predicted Material: {material.upper()}", material

def load_category_images(material):
    folder = os.path.join(BASE_PATH, material.lower())
    if not os.path.exists(folder):
        return [], [], []
    files = [f for f in os.listdir(folder) if f.lower().endswith(('.jpg','.jpeg','.png'))]
    soil = [os.path.join(folder,f) for f in files if 'soil' in f.lower()]
    hydro = [os.path.join(folder,f) for f in files if 'hydro' in f.lower()]
    aero = [os.path.join(folder,f) for f in files if 'aero' in f.lower()]
    return soil, hydro, aero

def get_instructions(material):
    general_instructions = """
### 🧭 General Instructions
- ♻️ Clean the waste material thoroughly before reuse.
- 🪚 Cut or modify safely using proper tools.
- 🌱 Choose a farming method (Soil-based, Hydroponics, Aeroponics).
- 💧 Maintain correct watering or nutrient levels.
- ☀️ Provide adequate sunlight or controlled temperature.
"""
    soil_instructions = """
### 🌱 Soil-based Farming
1. Fill with soil and compost mix.
2. Add sapling and ensure drainage holes.
3. Place under sunlight and water regularly.
"""
    hydro_instructions = """
### 💧 Hydroponics
1. Fill partially with nutrient solution.
2. Support roots using cocopeat or sponge.
3. Maintain aeration and pH balance.
"""
    aero_instructions = """
### 🌬️ Aeroponics
1. Suspend roots in air and mist periodically.
2. Keep spray nozzles clean.
3. Maintain humidity and temperature.
"""
    return general_instructions, soil_instructions, hydro_instructions, aero_instructions

def on_upload(img_path):
    status, material = classify_uploaded_image(img_path)
    general, soil_inst, hydro_inst, aero_inst = get_instructions(material)
    if material is None:
        return status, general, "", "", "", [], [], [], "No selection yet."
    soil, hydro, aero = load_category_images(material)
    def to_gallery(paths):
        return [(p, os.path.basename(p)) for p in paths if os.path.exists(p)]
    return status, general, soil_inst, hydro_inst, aero_inst, to_gallery(soil), to_gallery(hydro), to_gallery(aero), "No selection yet."

def on_gallery_select(evt: gr.SelectData, gallery_data):
    try:
        selected_index = evt.index
        if gallery_data is None or len(gallery_data) == 0:
            return "⚠️ No images in gallery"
        if selected_index >= len(gallery_data):
            return "⚠️ Invalid selection"
        selected_item = gallery_data[selected_index]
        selected_path = selected_item[0]
        selected_name = os.path.basename(selected_path)
        return selected_name
    except Exception as e:
        return f"⚠️ Error: {str(e)}"

def confirm_selected(selected_name):
    if not selected_name or "⚠️" in selected_name or "No selection" in selected_name:
        return "⚠️ No design selected. Click a thumbnail first."
    return f"✅ You selected: **{selected_name}**"

# ============================================================================
# -------------------- SEEDLING FUNCTIONALITY --------------------------------
# ============================================================================

PLANTS = [
    {"name": "Tomato", "soil": True, "hydro": True, "aero": True},
    {"name": "Brinjal", "soil": True, "hydro": True, "aero": False},
    {"name": "Ladies Finger", "soil": True, "hydro": True, "aero": False},
    {"name": "Ridge gaurd", "soil": True, "hydro": False, "aero": False},
    {"name": "Snake gaurd", "soil": True, "hydro": False, "aero": False},
    {"name": "Bottle gaurd", "soil": False, "hydro": True, "aero": False},
    {"name": "Beans", "soil": True, "hydro": True, "aero": False},
    {"name": "Cluster beans", "soil": True, "hydro": False, "aero": False},
    {"name": "Corriander", "soil": True, "hydro": True, "aero": True},
    {"name": "Fenugreek", "soil": True, "hydro": True, "aero": True},
    {"name": "Cucumber", "soil": True, "hydro": False, "aero": True},
    {"name": "Spinach", "soil": False, "hydro": False, "aero": True},
    {"name": "Mint", "soil": False, "hydro": False, "aero": True},
    {"name": "Basil", "soil": True, "hydro": True, "aero": True},
    {"name": "Chilli", "soil": True, "hydro": True, "aero": False},
    {"name": "Lemon grass", "soil": True, "hydro": False, "aero": False},
    {"name": "Aloe vera", "soil": True, "hydro": False, "aero": False},
    {"name": "Lettuce", "soil": False, "hydro": True, "aero": True},
    {"name": "Celery", "soil": False, "hydro": True, "aero": False},
    {"name": "Tomato Cherry", "soil": False, "hydro": False, "aero": False},
    {"name": "Water Spinach", "soil": False, "hydro": True, "aero": True},
    {"name": "Thyme", "soil": False, "hydro": False, "aero": True},
    {"name": "Microgreens", "soil": False, "hydro": False, "aero": True}
]

SEED_PRICES = {
    "Tomato": "₹20-40 per packet", "Brinjal": "₹15-30 per packet",
    "Ladies Finger": "₹20-35 per packet", "Ridge gaurd": "₹25-40 per packet",
    "Snake gaurd": "₹25-40 per packet", "Bottle gaurd": "₹20-35 per packet",
    "Beans": "₹30-50 per packet", "Cluster beans": "₹20-35 per packet",
    "Corriander": "₹10-20 per packet", "Fenugreek": "₹10-20 per packet",
    "Cucumber": "₹20-40 per packet", "Spinach": "₹15-30 per packet",
    "Mint": "₹15-25 per packet", "Basil": "₹20-35 per packet",
    "Chilli": "₹25-45 per packet", "Lemon grass": "₹30-50 per packet",
    "Aloe vera": "₹40-80 per plant", "Lettuce": "₹20-40 per packet",
    "Celery": "₹30-50 per packet", "Water Spinach": "₹15-30 per packet",
    "Thyme": "₹25-45 per packet", "Microgreens": "₹50-100 per packet",
    "Tomato Cherry": "₹30-60 per packet"
}

GROWING_TIPS = {
    "Tomato": "Sow seeds 1cm deep in moist soil; Transplant after 3-4 weeks when seedlings are 6 inches tall",
    "Brinjal": "Start seeds indoors in seedling trays; Transfer to containers after 5-6 weeks",
    "Ladies Finger": "Direct sow seeds 2cm deep in warm soil; Space plants 12 inches apart",
    "Ridge gaurd": "Soak seeds overnight before planting; Provide vertical support for climbing",
    "Snake gaurd": "Plant seeds in nutrient-rich compost; Needs trellis or support structure",
    "Bottle gaurd": "Start in small pots and transplant; Requires strong support for heavy fruits",
    "Beans": "Sow directly 3-4cm deep in well-drained soil; Water regularly but avoid overwatering",
    "Cluster beans": "Plant seeds 2cm deep in rows; Thrives in warm weather with minimal care",
    "Corriander": "Scatter seeds lightly on soil surface; Keep moist until germination in 7-10 days",
    "Fenugreek": "Soak seeds for 12 hours before sowing; Harvest leaves in 3-4 weeks",
    "Cucumber": "Plant 2-3 seeds per hole 2cm deep; Keep soil consistently moist",
    "Spinach": "Sow seeds thinly in shallow furrows; Harvest outer leaves regularly",
    "Mint": "Use stem cuttings or seeds in moist soil; Grows rapidly and spreads quickly",
    "Basil": "Start seeds indoors or in warm spot; Pinch off flowers to promote leaf growth",
    "Chilli": "Start indoors 8-10 weeks before transplanting; Needs warm temperatures to fruit",
    "Lemon grass": "Plant stalks from grocery store in water; Transfer to soil after roots develop",
    "Aloe vera": "Use offsets or pups from mother plant; Needs minimal watering and well-draining soil",
    "Lettuce": "Sow seeds on surface, press lightly; Harvest leaves in 4-6 weeks",
    "Celery": "Start seeds indoors in trays; Transplant after 10-12 weeks",
    "Water Spinach": "Plant stem cuttings in water or moist soil; Grows quickly in warm conditions",
    "Thyme": "Surface sow seeds or use cuttings; Requires well-draining soil and sunlight",
    "Microgreens": "Densely scatter seeds on moist growing medium; Harvest in 7-14 days",
    "Tomato Cherry": "Sow like regular tomatoes but in smaller pots; Provide support for vining varieties"
}

def extract_farming_type(filename):
    if not filename or "No selection" in filename or "⚠️" in filename:
        return None
    filename_lower = filename.lower()
    if 'soil' in filename_lower:
        return 'soil'
    elif 'hydro' in filename_lower:
        return 'hydro'
    elif 'aero' in filename_lower:
        return 'aero'
    return None

def get_suitable_plants(farming_type):
    suitable = []
    for plant in PLANTS:
        if plant.get(farming_type, False):
            suitable.append(plant["name"])
    random.shuffle(suitable)
    return suitable[:5]

def load_seed_images_from_folder():
    if not os.path.exists(SEED_FOLDER):
        return []
    files = [f for f in os.listdir(SEED_FOLDER) if f.lower().endswith(('.jpg', '.jpeg', '.png','.webp'))]
    return [os.path.join(SEED_FOLDER, f) for f in files]

def match_plant_to_image(plant_name, all_image_paths):
    plant_lower = plant_name.lower().replace(" ", "").replace("_", "")
    for img_path in all_image_paths:
        img_name = os.path.basename(img_path).lower()
        img_base = os.path.splitext(img_name)[0].replace(" ", "").replace("_", "")
        if plant_lower in img_base or img_base in plant_lower:
            return img_path
    return None

def generate_seedling_info(selected_name):
    farming_type = extract_farming_type(selected_name)
    if not farming_type:
        return (gr.update(visible=False), "", None) * 5
    all_seed_images = load_seed_images_from_folder()
    if not all_seed_images:
        return (gr.update(visible=True), "⚠️ No seed images found", None) * 5
    plants = get_suitable_plants(farming_type)
    if not plants:
        return (gr.update(visible=True), "No suitable plants found", None) * 5
    outputs = []
    for i in range(5):
        if i < len(plants):
            plant_name = plants[i]
            img_path = match_plant_to_image(plant_name, all_seed_images)
            price = SEED_PRICES.get(plant_name, "₹20-50 per packet")
            tips = GROWING_TIPS.get(plant_name, "Sow seeds in moist soil; Keep in warm, sunny location")
            plant_text = f"## 🌿 {plant_name}\n\n**Growing Instructions:**\n- 🏡 {tips}\n- 💰 Market Price: {price}\n"
            outputs.extend([gr.update(visible=True), plant_text, img_path if img_path and os.path.exists(img_path) else None])
        else:
            outputs.extend([gr.update(visible=False), "", None])
    return tuple(outputs)

# ============================================================================
# -------------------- MANURE CREATION FUNCTIONALITY -------------------------
# ============================================================================

MANURE_INSTRUCTIONS = {
    "Curry leaves": {"prep": "Wash and chop curry leaves into small pieces", "fermentation": "Soak in water for 7-10 days in a covered container", "nutrients": "Rich in nitrogen, iron, and minerals. Excellent for leafy growth", "application": "Dilute 1:5 with water before applying to soil/roots"},
    "Apple cores": {"prep": "Chop apple cores into small pieces, remove seeds", "fermentation": "Soak in water for 5-7 days, stir daily", "nutrients": "Contains potassium, phosphorus, and natural sugars", "application": "Strain and dilute 1:4 with water. Use for flowering plants"},
    "Cucumber peel": {"prep": "Slice cucumber peels thinly for faster decomposition", "fermentation": "Ferment in water for 5-7 days in shade", "nutrients": "High in silica and potassium. Good for stem strength", "application": "Mix 1 part solution with 3 parts water"},
    "Tomato waste": {"prep": "Crush tomato waste to release nutrients quickly", "fermentation": "Steep in water for 7-10 days, covered", "nutrients": "Rich in phosphorus and calcium. Promotes fruiting", "application": "Dilute 1:5 and apply during flowering stage"},
    "Banana peel": {"prep": "Cut banana peels into 1-inch pieces", "fermentation": "Soak in water for 48-72 hours, stir twice daily", "nutrients": "Extremely high in potassium. Boosts flowering and fruiting", "application": "Dilute 1:3 with water. Excellent for all plants"},
    "Potato peel": {"prep": "Chop potato peels finely", "fermentation": "Ferment for 5-7 days in water", "nutrients": "Contains potassium and starch. Good for root development", "application": "Strain and dilute 1:4 before use"},
    "Onion peel": {"prep": "Collect dry outer peels and crush lightly", "fermentation": "Soak in warm water for 24 hours", "nutrients": "Contains sulfur and antioxidants. Natural pest deterrent", "application": "Use undiluted as spray for pest control and nutrition"},
    "Garlic peel": {"prep": "Crush garlic peels to release compounds", "fermentation": "Steep in water for 24-48 hours", "nutrients": "Rich in sulfur. Acts as natural fungicide and pest repellent", "application": "Dilute 1:2 and spray on leaves and soil"},
    "Tea waste": {"prep": "Collect used tea leaves/bags", "fermentation": "Soak in water for 3-5 days", "nutrients": "Contains nitrogen and tannic acid. Lowers pH for acid-loving plants", "application": "Dilute 1:4 with water"},
    "Eggshells": {"prep": "Crush eggshells into fine powder", "fermentation": "Soak in vinegar-water (1:10) for 7-14 days", "nutrients": "Excellent source of calcium. Prevents blossom end rot", "application": "Dilute 1:5 and apply to soil around plants"},
    "Milk waste (sour milk, curd)": {"prep": "Dilute sour milk or curd with water immediately", "fermentation": "Let stand for 24-48 hours", "nutrients": "Rich in calcium, proteins, and beneficial bacteria", "application": "Dilute 1:4 with water. Acts as mild fungicide"},
    "Rice water": {"prep": "Collect water after washing rice (first wash preferred)", "fermentation": "Ferment for 3-5 days until slightly sour", "nutrients": "Contains NPK and promotes beneficial microorganisms", "application": "Dilute 1:2 with water. Excellent for all plants"},
    "Pomegranate peel": {"prep": "Dry peels in sun for 2 days, then powder them", "fermentation": "Soak powder in water for 7-10 days", "nutrients": "High in antioxidants and minerals. Pest deterrent", "application": "Strain and dilute 1:5 before applying"},
    "Mango peel": {"prep": "Chop mango peels into small pieces", "fermentation": "Ferment in water for 7-10 days with occasional stirring", "nutrients": "Rich in vitamins and minerals. Good for overall plant health", "application": "Dilute 1:4 and use during vegetative growth"},
    "Coconut husk": {"prep": "Break coconut husk into small pieces or powder", "fermentation": "Soak in water for 14-21 days (longer fermentation)", "nutrients": "High in potassium and lignin. Improves soil structure", "application": "Dilute 1:3 with water"},
    "Ginger peel waste": {"prep": "Chop ginger peels finely", "fermentation": "Steep in water for 5-7 days", "nutrients": "Contains minerals and natural antibacterial compounds", "application": "Dilute 1:4. Also acts as natural pest repellent"},
    "Cabbage outer leaves": {"prep": "Chop cabbage leaves into small pieces", "fermentation": "Soak in water for 7-10 days", "nutrients": "Rich in nitrogen and calcium. Good for leafy vegetables", "application": "Dilute 1:5 before applying to soil"},
    "Spinach stems": {"prep": "Chop spinach stems finely", "fermentation": "Ferment in water for 5-7 days", "nutrients": "High in iron and nitrogen. Excellent for leafy growth", "application": "Dilute 1:4 with water"}
}

def generate_manure_combinations():
    soil_combinations = [
        ["Banana peel"], ["Rice water"], ["Tea waste"], ["Eggshells"], ["Curry leaves"],
        ["Tomato waste"], ["Apple cores"], ["Milk waste (sour milk, curd)"], ["Onion peel"], ["Garlic peel"]
    ]
    hydro_combinations = [
        ["Banana peel", "Rice water"], ["Tea waste", "Eggshells"], ["Tomato waste", "Milk waste (sour milk, curd)"],
        ["Cucumber peel", "Rice water"], ["Apple cores", "Banana peel"], ["Onion peel", "Garlic peel"],
        ["Curry leaves", "Rice water"], ["Potato peel", "Tea waste"], ["Spinach stems", "Eggshells"], ["Mango peel", "Rice water"]
    ]
    aero_combinations = [
        ["Banana peel", "Rice water", "Tea waste"], ["Eggshells", "Tomato waste", "Milk waste (sour milk, curd)"],
        ["Cucumber peel", "Apple cores", "Rice water"], ["Onion peel", "Garlic peel", "Curry leaves"],
        ["Potato peel", "Banana peel", "Tea waste"], ["Spinach stems", "Cabbage outer leaves", "Rice water"],
        ["Mango peel", "Pomegranate peel", "Rice water"], ["Ginger peel waste", "Garlic peel", "Onion peel"],
        ["Coconut husk", "Banana peel", "Rice water"], ["Apple cores", "Tomato waste", "Eggshells"]
    ]
    return soil_combinations, hydro_combinations, aero_combinations

def get_manure_recommendations(selected_name):
    farming_type = extract_farming_type(selected_name)
    if not farming_type:
        return (gr.update(visible=False), "") * 4
    soil_combos, hydro_combos, aero_combos = generate_manure_combinations()
    if farming_type == 'soil':
        selected_combos = random.sample(soil_combos, 4)
    elif farming_type == 'hydro':
        selected_combos = random.sample(hydro_combos, 4)
    else:
        selected_combos = random.sample(aero_combos, 4)
    outputs = []
    for i, combo in enumerate(selected_combos, 1):
        combo_text = f"## 🧪 Liquid Manure Recipe {i}\n\n### Ingredients:\n"
        for ingredient in combo:
            combo_text += f"- **{ingredient}**\n"
        combo_text += "\n### Detailed Instructions:\n\n"
        for ingredient in combo:
            if ingredient in MANURE_INSTRUCTIONS:
                inst = MANURE_INSTRUCTIONS[ingredient]
                combo_text += f"**{ingredient}:**\n- 🔪 Preparation: {inst['prep']}\n- ⏱️ Fermentation: {inst['fermentation']}\n- 🌱 Nutrients: {inst['nutrients']}\n- 💧 Application: {inst['application']}\n\n"
        if len(combo) == 1:
            combo_text += "### Final Steps:\n1. Use a clean plastic container for fermentation\n2. Cover with a breathable cloth to allow gases to escape\n3. Stir once daily during fermentation\n4. Strain before use\n5. Store in a cool, dark place for up to 2 weeks\n"
        elif len(combo) == 2:
            combo_text += "### Final Steps:\n1. Prepare both ingredients separately as per instructions above\n2. After individual fermentation, mix the two solutions in equal parts\n3. Let the mixture settle for 24 hours\n4. Dilute the final mixture 1:5 with water before application\n5. Use within 1 week for best results\n"
        else:
            combo_text += "### Final Steps:\n1. Prepare all three ingredients separately as per instructions above\n2. After fermentation, combine all three solutions in equal parts\n3. Mix thoroughly and let it rest for 48 hours\n4. Dilute the final mixture 1:6 with water before application\n5. For aeroponics, ensure the solution is well-strained to prevent nozzle clogging\n6. Use within 5 days for optimal nutrient content\n"
        combo_text += "\n---\n"
        outputs.extend([gr.update(visible=True), combo_text])
    return tuple(outputs)

# ============================================================================
# -------------------- UI LAYOUT WITH TABS -----------------------------------
# ============================================================================

# -------------------- UI LAYOUT WITH TABS -----------------------------------
# ============================================================================

with gr.Blocks(theme=gr.themes.Soft(primary_hue="green"), title="Sprout Smart") as app:
    # MODIFIED: Centered title with larger font
    gr.Markdown(
        """
        <h1 style='text-align: center; font-size: 48px; font-weight: bold; color: #2d5016; margin-bottom: 10px;'>
        ♻️ Sprout Smart
        </h1>
        <p style='text-align: center; font-size: 18px; color: #5a7c3e;'>
        Transform plastic waste into farming containers and discover the perfect plants to grow!
        </p>
        """,
        elem_id="main-title"
    )
    with gr.Tabs():
        # TAB 1: RESTRUCTURING
        with gr.Tab("♻️ Restructuring"):
            gr.Markdown("Upload an image of a **plastic waste** item to classify it (bottle/container).\nThen explore eco-friendly **reuse ideas** under Soil, Hydro, and Aeroponics 🌱")
            with gr.Row():
                upload = gr.Image(label="📤 Upload Material", type="filepath", height=160)
                status = gr.Textbox(label="Prediction", interactive=False)
            general_box = gr.Markdown("Upload an image to get started.")
            gr.Markdown("---")
            soil_inst_box = gr.Markdown("")
            soil_gallery = gr.Gallery(columns=6, height=150, label="Soil-based Farming", object_fit="cover", container=False, show_label=True)
            gr.Markdown("---")
            hydro_inst_box = gr.Markdown("")
            hydro_gallery = gr.Gallery(columns=6, height=150, label="Hydroponics", object_fit="cover", container=False, show_label=True)
            gr.Markdown("---")
            aero_inst_box = gr.Markdown("")
            aero_gallery = gr.Gallery(columns=6, height=150, label="Aeroponics", object_fit="cover", container=False, show_label=True)
            gr.Markdown("---")
            with gr.Row():
                choose_btn = gr.Button("Select this design")
                selected_text = gr.Markdown("No selection yet.")
            upload.change(on_upload, inputs=upload, outputs=[status, general_box, soil_inst_box, hydro_inst_box, aero_inst_box, soil_gallery, hydro_gallery, aero_gallery, selected_text])
            soil_gallery.select(on_gallery_select, inputs=[soil_gallery], outputs=[selected_text])
            hydro_gallery.select(on_gallery_select, inputs=[hydro_gallery], outputs=[selected_text])
            aero_gallery.select(on_gallery_select, inputs=[aero_gallery], outputs=[selected_text])
            choose_btn.click(confirm_selected, inputs=selected_text, outputs=selected_text)

        # TAB 2: SEEDLINGS
        with gr.Tab("🌱 Seedlings"):
            gr.Markdown("## Get Personalized Plant Recommendations\nAfter selecting a restructuring design in the previous tab, come here to discover which plants are perfect for your chosen farming method!")
            gr.Markdown("---")
            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("### How to use:\n1. Select a design in the **Restructuring** tab\n2. Come back to this tab\n3. Click \"Show Seedlings\" below\n4. Get 5 recommended plants with images and growing tips!")
                    seedling_btn = gr.Button("🌿 Show Seedlings", variant="primary", size="lg")
            gr.Markdown("---")
            gr.Markdown("# 🌱 Recommended Plants for Your Farming Method")
            plant_blocks = []
            for i in range(5):
                with gr.Column(visible=False) as plant_col:
                    plant_text = gr.Markdown("")
                    plant_img = gr.Image(label=f"Plant {i+1}", height=250, show_label=False)
                    gr.Markdown("---")
                plant_blocks.append((plant_col, plant_text, plant_img))
            all_plant_outputs = []
            for col, text, img in plant_blocks:
                all_plant_outputs.extend([col, text, img])
            seedling_btn.click(generate_seedling_info, inputs=selected_text, outputs=all_plant_outputs)

        # TAB 3: MANURE CREATION
        with gr.Tab("🍂 Manure Creation"):
            gr.Markdown("## Create Organic Liquid Manure\nBased on your selected farming method, get customized recipes using kitchen waste to create nutrient-rich liquid fertilizer!")
            gr.Markdown("---")
            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("### How to use:\n1. Select a design in the **Restructuring** tab\n2. Come back to this tab\n3. Click \"Show Manure Recipes\" below\n4. Get 4 organic manure recipes tailored to your farming method!")
                    manure_btn = gr.Button("🧪 Show Manure Recipes", variant="primary", size="lg")
            gr.Markdown("---")
            gr.Markdown("# 🧪 Organic Liquid Manure Recipes")
            manure_blocks = []
            for i in range(4):
                with gr.Column(visible=False) as manure_col:
                    manure_text = gr.Markdown("")
                    gr.Markdown("---")
                manure_blocks.append((manure_col, manure_text))
            all_manure_outputs = []
            for col, text in manure_blocks:
                all_manure_outputs.extend([col, text])
            manure_btn.click(get_manure_recommendations, inputs=selected_text, outputs=all_manure_outputs)

app.launch(share=True)

✅ Model loaded successfully!


/usr/local/lib/python3.12/dist-packages/gradio/components/base.py:200: UserWarning: show_label has no effect when container is False.
  warnings.warn("show_label has no effect when container is False.")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fe0ec3c3ada5118028.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install gradio --quiet
import gradio as gr
import os
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model
import random

# ---------------- CONFIG ----------------
MODEL_PATH = "/content/drive/MyDrive/plastic_classifier_model.h5"
BASE_PATH = "/content/drive/MyDrive/restructure_material"
SEED_FOLDER = "/content/drive/MyDrive/plant_seeds"
MANURE_FOLDER = "/content/drive/MyDrive/manure"  # NEW: Add this folder for manure images
PEST_FOLDER = "/content/drive/MyDrive/pest_control"
PEST_DISEASE_FOLDER = "/content/drive/MyDrive/pest_diseases"# Folder containing pest control ingredient images

# ---------------- MODEL ----------------
try:
    model = load_model(MODEL_PATH, compile=False)
    print("✅ Model loaded successfully!")
except Exception as e:
    print("⚠️ Model load failed, using dummy:", e)
    class Dummy:
        def predict(self, x): return np.array([[0.9, 0.1]])
    model = Dummy()

# ---------------- HELPERS (RESTRUCTURING) ----------------
def classify_uploaded_image(uploaded_path):
    if not uploaded_path:
        return "⚠️ Upload an image first.", None
    img = Image.open(uploaded_path).convert("RGB").resize((160,160))
    arr = np.expand_dims(np.array(img)/255., axis=0)
    pred = model.predict(arr)
    pred = np.array(pred).squeeze()
    if pred.size == 1:
        idx = 0 if pred < 0.5 else 1
        conf = float(pred)*100
    else:
        idx = int(np.argmax(pred))
        conf = float(np.max(pred))*100
    material = "bottle" if idx==0 else "container"
    return f"✅ Predicted Material: {material.upper()}", material

def load_category_images(material):
    folder = os.path.join(BASE_PATH, material.lower())
    if not os.path.exists(folder):
        return [], [], []
    files = [f for f in os.listdir(folder) if f.lower().endswith(('.jpg','.jpeg','.png'))]
    soil = [os.path.join(folder,f) for f in files if 'soil' in f.lower()]
    hydro = [os.path.join(folder,f) for f in files if 'hydro' in f.lower()]
    aero = [os.path.join(folder,f) for f in files if 'aero' in f.lower()]
    return soil, hydro, aero

def get_instructions(material):
    general_instructions = """
### 🧭 General Instructions
- ♻️ Clean the waste material thoroughly before reuse.
- 🪚 Cut or modify safely using proper tools.
- 🌱 Choose a farming method (Soil-based, Hydroponics, Aeroponics).
- 💧 Maintain correct watering or nutrient levels.
- ☀️ Provide adequate sunlight or controlled temperature.
"""
    soil_instructions = """
### 🌱 Soil-based Farming
1. Fill with soil and compost mix.
2. Add sapling and ensure drainage holes.
3. Place under sunlight and water regularly.
"""
    hydro_instructions = """
### 💧 Hydroponics
1. Fill partially with nutrient solution.
2. Support roots using cocopeat or sponge.
3. Maintain aeration and pH balance.
"""
    aero_instructions = """
### 🌬️ Aeroponics
1. Suspend roots in air and mist periodically.
2. Keep spray nozzles clean.
3. Maintain humidity and temperature.
"""
    return general_instructions, soil_instructions, hydro_instructions, aero_instructions

def on_upload(img_path):
    status, material = classify_uploaded_image(img_path)
    general, soil_inst, hydro_inst, aero_inst = get_instructions(material)
    if material is None:
        return status, general, "", "", "", [], [], [], "No selection yet."
    soil, hydro, aero = load_category_images(material)
    def to_gallery(paths):
        return [(p, os.path.basename(p)) for p in paths if os.path.exists(p)]
    return status, general, soil_inst, hydro_inst, aero_inst, to_gallery(soil), to_gallery(hydro), to_gallery(aero), "No selection yet."

def on_gallery_select(evt: gr.SelectData, gallery_data):
    try:
        selected_index = evt.index
        if gallery_data is None or len(gallery_data) == 0:
            return "⚠️ No images in gallery"
        if selected_index >= len(gallery_data):
            return "⚠️ Invalid selection"
        selected_item = gallery_data[selected_index]
        selected_path = selected_item[0]
        selected_name = os.path.basename(selected_path)
        return selected_name
    except Exception as e:
        return f"⚠️ Error: {str(e)}"

def confirm_selected(selected_name):
    if not selected_name or "⚠️" in selected_name or "No selection" in selected_name:
        return "⚠️ No design selected. Click a thumbnail first."
    return f"✅ You selected: **{selected_name}**"

# ============================================================================
# -------------------- SEEDLING FUNCTIONALITY --------------------------------
# ============================================================================

PLANTS = [
    {"name": "Tomato", "soil": True, "hydro": True, "aero": True},
    {"name": "Brinjal", "soil": True, "hydro": True, "aero": False},
    {"name": "Ladies Finger", "soil": True, "hydro": True, "aero": False},
    {"name": "Ridge gaurd", "soil": True, "hydro": False, "aero": False},
    {"name": "Snake gaurd", "soil": True, "hydro": False, "aero": False},
    {"name": "Bottle gaurd", "soil": False, "hydro": True, "aero": False},
    {"name": "Beans", "soil": True, "hydro": True, "aero": False},
    {"name": "Cluster beans", "soil": True, "hydro": False, "aero": False},
    {"name": "Corriander", "soil": True, "hydro": True, "aero": True},
    {"name": "Fenugreek", "soil": True, "hydro": True, "aero": True},
    {"name": "Cucumber", "soil": True, "hydro": False, "aero": True},
    {"name": "Spinach", "soil": False, "hydro": False, "aero": True},
    {"name": "Mint", "soil": False, "hydro": False, "aero": True},
    {"name": "Basil", "soil": True, "hydro": True, "aero": True},
    {"name": "Chilli", "soil": True, "hydro": True, "aero": False},
    {"name": "Lemon grass", "soil": True, "hydro": False, "aero": False},
    {"name": "Aloe vera", "soil": True, "hydro": False, "aero": False},
    {"name": "Lettuce", "soil": False, "hydro": True, "aero": True},
    {"name": "Celery", "soil": False, "hydro": True, "aero": False},
    {"name": "Tomato Cherry", "soil": False, "hydro": False, "aero": False},
    {"name": "Water Spinach", "soil": False, "hydro": True, "aero": True},
    {"name": "Thyme", "soil": False, "hydro": False, "aero": True},
    {"name": "Microgreens", "soil": False, "hydro": False, "aero": True}
]

SEED_PRICES = {
    "Tomato": "₹20-40 per packet", "Brinjal": "₹15-30 per packet",
    "Ladies Finger": "₹20-35 per packet", "Ridge gaurd": "₹25-40 per packet",
    "Snake gaurd": "₹25-40 per packet", "Bottle gaurd": "₹20-35 per packet",
    "Beans": "₹30-50 per packet", "Cluster beans": "₹20-35 per packet",
    "Corriander": "₹10-20 per packet", "Fenugreek": "₹10-20 per packet",
    "Cucumber": "₹20-40 per packet", "Spinach": "₹15-30 per packet",
    "Mint": "₹15-25 per packet", "Basil": "₹20-35 per packet",
    "Chilli": "₹25-45 per packet", "Lemon grass": "₹30-50 per packet",
    "Aloe vera": "₹40-80 per plant", "Lettuce": "₹20-40 per packet",
    "Celery": "₹30-50 per packet", "Water Spinach": "₹15-30 per packet",
    "Thyme": "₹25-45 per packet", "Microgreens": "₹50-100 per packet",
    "Tomato Cherry": "₹30-60 per packet"
}

GROWING_TIPS = {
    "Tomato": "Sow seeds 1cm deep in moist soil; Transplant after 3-4 weeks when seedlings are 6 inches tall",
    "Brinjal": "Start seeds indoors in seedling trays; Transfer to containers after 5-6 weeks",
    "Ladies Finger": "Direct sow seeds 2cm deep in warm soil; Space plants 12 inches apart",
    "Ridge gaurd": "Soak seeds overnight before planting; Provide vertical support for climbing",
    "Snake gaurd": "Plant seeds in nutrient-rich compost; Needs trellis or support structure",
    "Bottle gaurd": "Start in small pots and transplant; Requires strong support for heavy fruits",
    "Beans": "Sow directly 3-4cm deep in well-drained soil; Water regularly but avoid overwatering",
    "Cluster beans": "Plant seeds 2cm deep in rows; Thrives in warm weather with minimal care",
    "Corriander": "Scatter seeds lightly on soil surface; Keep moist until germination in 7-10 days",
    "Fenugreek": "Soak seeds for 12 hours before sowing; Harvest leaves in 3-4 weeks",
    "Cucumber": "Plant 2-3 seeds per hole 2cm deep; Keep soil consistently moist",
    "Spinach": "Sow seeds thinly in shallow furrows; Harvest outer leaves regularly",
    "Mint": "Use stem cuttings or seeds in moist soil; Grows rapidly and spreads quickly",
    "Basil": "Start seeds indoors or in warm spot; Pinch off flowers to promote leaf growth",
    "Chilli": "Start indoors 8-10 weeks before transplanting; Needs warm temperatures to fruit",
    "Lemon grass": "Plant stalks from grocery store in water; Transfer to soil after roots develop",
    "Aloe vera": "Use offsets or pups from mother plant; Needs minimal watering and well-draining soil",
    "Lettuce": "Sow seeds on surface, press lightly; Harvest leaves in 4-6 weeks",
    "Celery": "Start seeds indoors in trays; Transplant after 10-12 weeks",
    "Water Spinach": "Plant stem cuttings in water or moist soil; Grows quickly in warm conditions",
    "Thyme": "Surface sow seeds or use cuttings; Requires well-draining soil and sunlight",
    "Microgreens": "Densely scatter seeds on moist growing medium; Harvest in 7-14 days",
    "Tomato Cherry": "Sow like regular tomatoes but in smaller pots; Provide support for vining varieties"
}

def extract_farming_type(filename):
    if not filename or "No selection" in filename or "⚠️" in filename:
        return None
    filename_lower = filename.lower()
    if 'soil' in filename_lower:
        return 'soil'
    elif 'hydro' in filename_lower:
        return 'hydro'
    elif 'aero' in filename_lower:
        return 'aero'
    return None

def get_suitable_plants(farming_type):
    suitable = []
    for plant in PLANTS:
        if plant.get(farming_type, False):
            suitable.append(plant["name"])
    random.shuffle(suitable)
    return suitable[:5]

def load_seed_images_from_folder():
    if not os.path.exists(SEED_FOLDER):
        return []
    files = [f for f in os.listdir(SEED_FOLDER) if f.lower().endswith(('.jpg', '.jpeg', '.png','.webp'))]
    return [os.path.join(SEED_FOLDER, f) for f in files]

def match_plant_to_image(plant_name, all_image_paths):
    plant_lower = plant_name.lower().replace(" ", "").replace("_", "")
    for img_path in all_image_paths:
        img_name = os.path.basename(img_path).lower()
        img_base = os.path.splitext(img_name)[0].replace(" ", "").replace("_", "")
        if plant_lower in img_base or img_base in plant_lower:
            return img_path
    return None

def generate_seedling_info(selected_name):
    farming_type = extract_farming_type(selected_name)
    if not farming_type:
        return (gr.update(visible=False), "", None) * 5
    all_seed_images = load_seed_images_from_folder()
    if not all_seed_images:
        return (gr.update(visible=True), "⚠️ No seed images found", None) * 5
    plants = get_suitable_plants(farming_type)
    if not plants:
        return (gr.update(visible=True), "No suitable plants found", None) * 5
    outputs = []
    for i in range(5):
        if i < len(plants):
            plant_name = plants[i]
            img_path = match_plant_to_image(plant_name, all_seed_images)
            price = SEED_PRICES.get(plant_name, "₹20-50 per packet")
            tips = GROWING_TIPS.get(plant_name, "Sow seeds in moist soil; Keep in warm, sunny location")
            plant_text = f"## 🌿 {plant_name}\n\n**Growing Instructions:**\n- 🏡 {tips}\n- 💰 Market Price: {price}\n"
            outputs.extend([gr.update(visible=True), plant_text, img_path if img_path and os.path.exists(img_path) else None])
        else:
            outputs.extend([gr.update(visible=False), "", None])
    return tuple(outputs)

# ============================================================================
# -------------------- MANURE CREATION FUNCTIONALITY -------------------------
# ============================================================================

MANURE_INSTRUCTIONS = {
    "Curry leaves": {"prep": "Wash and chop curry leaves into small pieces", "fermentation": "Soak in water for 7-10 days in a covered container", "nutrients": "Rich in nitrogen, iron, and minerals. Excellent for leafy growth", "application": "Dilute 1:5 with water before applying to soil/roots"},
    "Apple cores": {"prep": "Chop apple cores into small pieces, remove seeds", "fermentation": "Soak in water for 5-7 days, stir daily", "nutrients": "Contains potassium, phosphorus, and natural sugars", "application": "Strain and dilute 1:4 with water. Use for flowering plants"},
    "Cucumber peel": {"prep": "Slice cucumber peels thinly for faster decomposition", "fermentation": "Ferment in water for 5-7 days in shade", "nutrients": "High in silica and potassium. Good for stem strength", "application": "Mix 1 part solution with 3 parts water"},
    "Tomato waste": {"prep": "Crush tomato waste to release nutrients quickly", "fermentation": "Steep in water for 7-10 days, covered", "nutrients": "Rich in phosphorus and calcium. Promotes fruiting", "application": "Dilute 1:5 and apply during flowering stage"},
    "Banana peel": {"prep": "Cut banana peels into 1-inch pieces", "fermentation": "Soak in water for 48-72 hours, stir twice daily", "nutrients": "Extremely high in potassium. Boosts flowering and fruiting", "application": "Dilute 1:3 with water. Excellent for all plants"},
    "Potato peel": {"prep": "Chop potato peels finely", "fermentation": "Ferment for 5-7 days in water", "nutrients": "Contains potassium and starch. Good for root development", "application": "Strain and dilute 1:4 before use"},
    "Onion peel": {"prep": "Collect dry outer peels and crush lightly", "fermentation": "Soak in warm water for 24 hours", "nutrients": "Contains sulfur and antioxidants. Natural pest deterrent", "application": "Use undiluted as spray for pest control and nutrition"},
    "Garlic peel": {"prep": "Crush garlic peels to release compounds", "fermentation": "Steep in water for 24-48 hours", "nutrients": "Rich in sulfur. Acts as natural fungicide and pest repellent", "application": "Dilute 1:2 and spray on leaves and soil"},
    "Tea waste": {"prep": "Collect used tea leaves/bags", "fermentation": "Soak in water for 3-5 days", "nutrients": "Contains nitrogen and tannic acid. Lowers pH for acid-loving plants", "application": "Dilute 1:4 with water"},
    "Eggshells": {"prep": "Crush eggshells into fine powder", "fermentation": "Soak in vinegar-water (1:10) for 7-14 days", "nutrients": "Excellent source of calcium. Prevents blossom end rot", "application": "Dilute 1:5 and apply to soil around plants"},
    "Milk waste": {"prep": "Dilute sour milk or curd with water immediately", "fermentation": "Let stand for 24-48 hours", "nutrients": "Rich in calcium, proteins, and beneficial bacteria", "application": "Dilute 1:4 with water. Acts as mild fungicide"},
    "Rice water": {"prep": "Collect water after washing rice (first wash preferred)", "fermentation": "Ferment for 3-5 days until slightly sour", "nutrients": "Contains NPK and promotes beneficial microorganisms", "application": "Dilute 1:2 with water. Excellent for all plants"},
    "Pomegranate peel": {"prep": "Dry peels in sun for 2 days, then powder them", "fermentation": "Soak powder in water for 7-10 days", "nutrients": "High in antioxidants and minerals. Pest deterrent", "application": "Strain and dilute 1:5 before applying"},
    "Mango peel": {"prep": "Chop mango peels into small pieces", "fermentation": "Ferment in water for 7-10 days with occasional stirring", "nutrients": "Rich in vitamins and minerals. Good for overall plant health", "application": "Dilute 1:4 and use during vegetative growth"},
    "Coconut husk": {"prep": "Break coconut husk into small pieces or powder", "fermentation": "Soak in water for 14-21 days (longer fermentation)", "nutrients": "High in potassium and lignin. Improves soil structure", "application": "Dilute 1:3 with water"},
    "Ginger peel waste": {"prep": "Chop ginger peels finely", "fermentation": "Steep in water for 5-7 days", "nutrients": "Contains minerals and natural antibacterial compounds", "application": "Dilute 1:4. Also acts as natural pest repellent"},
    "Cabbage outer leaves": {"prep": "Chop cabbage leaves into small pieces", "fermentation": "Soak in water for 7-10 days", "nutrients": "Rich in nitrogen and calcium. Good for leafy vegetables", "application": "Dilute 1:5 before applying to soil"},
    "Spinach stems": {"prep": "Chop spinach stems finely", "fermentation": "Ferment in water for 5-7 days", "nutrients": "High in iron and nitrogen. Excellent for leafy growth", "application": "Dilute 1:4 with water"}
}

def generate_manure_combinations():
    soil_combinations = [
        ["Banana peel"], ["Rice water"], ["Tea waste"], ["Eggshells"], ["Curry leaves"],
        ["Tomato waste"], ["Apple cores"], ["Milk waste"], ["Onion peel"], ["Garlic peel"]
    ]
    hydro_combinations = [
        ["Banana peel", "Rice water"], ["Tea waste", "Eggshells"], ["Tomato waste", "Milk waste"],
        ["Cucumber peel", "Rice water"], ["Apple cores", "Banana peel"], ["Onion peel", "Garlic peel"],
        ["Curry leaves", "Rice water"], ["Potato peel", "Tea waste"], ["Spinach stems", "Eggshells"], ["Mango peel", "Rice water"]
    ]
    aero_combinations = [
        ["Banana peel", "Rice water", "Tea waste"], ["Eggshells", "Tomato waste", "Milk waste"],
        ["Cucumber peel", "Apple cores", "Rice water"], ["Onion peel", "Garlic peel", "Curry leaves"],
        ["Potato peel", "Banana peel", "Tea waste"], ["Spinach stems", "Cabbage outer leaves", "Rice water"],
        ["Mango peel", "Pomegranate peel", "Rice water"], ["Ginger peel waste", "Garlic peel", "Onion peel"],
        ["Coconut husk", "Banana peel", "Rice water"], ["Apple cores", "Tomato waste", "Eggshells"]
    ]
    return soil_combinations, hydro_combinations, aero_combinations

# NEW FUNCTION: Load manure ingredient images from folder
def load_manure_images_from_folder():
    if not os.path.exists(MANURE_FOLDER):
        return []
    files = [f for f in os.listdir(MANURE_FOLDER) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.webp'))]
    return [os.path.join(MANURE_FOLDER, f) for f in files]

# NEW FUNCTION: Match ingredient name to image (similar to plant matching)
def match_ingredient_to_image(ingredient_name, all_image_paths):
    ingredient_lower = ingredient_name.lower().replace(" ", "").replace("_", "").replace("(", "").replace(")", "").replace(",", "")
    for img_path in all_image_paths:
        img_name = os.path.basename(img_path).lower()
        img_base = os.path.splitext(img_name)[0].replace(" ", "").replace("_", "")
        if ingredient_lower in img_base or img_base in ingredient_lower:
            return img_path
    return None

# UPDATED FUNCTION: Get manure recommendations with images (FIXED VERSION - LIKE RESTRUCTURING)
# UPDATED FUNCTION: Get manure recommendations with images (FIXED VERSION - LIKE RESTRUCTURING)
def get_manure_recommendations(selected_name):
    farming_type = extract_farming_type(selected_name)
    if not farming_type:
        return (gr.update(visible=False), "", [], "") * 4  # 4 recipes × 4 outputs each (visibility, heading, gallery, instructions)

    # Load all manure ingredient images
    all_manure_images = load_manure_images_from_folder()

    soil_combos, hydro_combos, aero_combos = generate_manure_combinations()
    if farming_type == 'soil':
        selected_combos = random.sample(soil_combos, 4)
    elif farming_type == 'hydro':
        selected_combos = random.sample(hydro_combos, 4)
    else:
        selected_combos = random.sample(aero_combos, 4)

    outputs = []
    for i, combo in enumerate(selected_combos, 1):
        # HEADING + INGREDIENTS
        heading_text = f"## 🧪 Liquid Manure Recipe {i}\n\n### Ingredients:\n"
        for ingredient in combo:
            heading_text += f"- **{ingredient}**\n"

        # DETAILED INSTRUCTIONS (separate from heading)
        instruction_text = "### Detailed Instructions:\n\n"
        for ingredient in combo:
            if ingredient in MANURE_INSTRUCTIONS:
                inst = MANURE_INSTRUCTIONS[ingredient]
                instruction_text += f"**{ingredient}:**\n- 🔪 Preparation: {inst['prep']}\n- ⏱️ Fermentation: {inst['fermentation']}\n- 🌱 Nutrients: {inst['nutrients']}\n- 💧 Application: {inst['application']}\n\n"

        if len(combo) == 1:
            instruction_text += "### Final Steps:\n1. Use a clean plastic container for fermentation\n2. Cover with a breathable cloth to allow gases to escape\n3. Stir once daily during fermentation\n4. Strain before use\n5. Store in a cool, dark place for up to 2 weeks\n"
        elif len(combo) == 2:
            instruction_text += "### Final Steps:\n1. Prepare both ingredients separately as per instructions above\n2. After individual fermentation, mix the two solutions in equal parts\n3. Let the mixture settle for 24 hours\n4. Dilute the final mixture 1:5 with water before application\n5. Use within 1 week for best results\n"
        else:
            instruction_text += "### Final Steps:\n1. Prepare all three ingredients separately as per instructions above\n2. After fermentation, combine all three solutions in equal parts\n3. Mix thoroughly and let it rest for 48 hours\n4. Dilute the final mixture 1:6 with water before application\n5. For aeroponics, ensure the solution is well-strained to prevent nozzle clogging\n6. Use within 5 days for optimal nutrient content\n"

        # FIXED: Find images for ALL ingredients in the combo (like restructuring part)
        img_paths = []
        if all_manure_images and combo:
            for ingredient in combo:
                img_path = match_ingredient_to_image(ingredient, all_manure_images)
                if img_path and os.path.exists(img_path):
                    img_paths.append(img_path)

        # Convert to gallery format (like in restructuring: to_gallery function)
        def to_gallery(paths):
            return [(p, os.path.basename(p)) for p in paths if os.path.exists(p)]

        gallery_data = to_gallery(img_paths)

        # Output: [visibility, heading_with_ingredients, gallery, detailed_instructions]
        outputs.extend([gr.update(visible=True), heading_text, gallery_data, instruction_text])

    return tuple(outputs)
def load_pest_images_from_folder():
    if not os.path.exists(PEST_FOLDER):
        return []
    files = [f for f in os.listdir(PEST_FOLDER) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.webp'))]
    return [os.path.join(PEST_FOLDER, f) for f in files]

# MATCH PEST CONTROL SOLUTION TO IMAGE
def match_pest_to_image(solution_name, all_image_paths):
    solution_lower = solution_name.lower().replace(" ", "").replace("_", "").replace("(", "").replace(")", "").replace("-", "")
    for img_path in all_image_paths:
        img_name = os.path.basename(img_path).lower()
        img_base = os.path.splitext(img_name)[0].replace(" ", "").replace("_", "").replace("-", "")
        if solution_lower in img_base or img_base in solution_lower:
            return img_path
        # Also check individual words for matches
        solution_words = solution_name.lower().split()
        for word in solution_words:
            if len(word) > 3 and word in img_base:
                return img_path
    return None

# PEST CONTROL INSTRUCTIONS DICTIONARY
PEST_CONTROL_INSTRUCTIONS = {
    # SOIL-BASED SOLUTIONS
    "Garlic-Onion Spray": {
        "source": "Garlic, onion peels, chili",
        "target": "Aphids, beetles, leaf spot fungus",
        "how_it_works": "Allicin and sulfur act as natural antibiotics and repellents",
        "preparation": "Crush 5-6 garlic cloves, 1 onion, and 2 chilies. Steep in 1 liter water for 24 hours",
        "application": "Strain and spray on affected leaves every 3-4 days. Best applied in early morning"
    },
    "Neem Compost Tea": {
        "source": "Neem seeds + kitchen compost",
        "target": "Aphids",
        "how_it_works": "Azadirachtin in neem regulates pest growth and disrupts feeding",
        "preparation": "Soak 100g crushed neem seeds + 200g compost in 2 liters water for 48 hours",
        "application": "Dilute 1:3 with water. Apply to soil around plant base weekly"
    },
    "Ginger-Turmeric Extract": {
        "source": "Ginger & turmeric peels",
        "target": "Fungal wilt, bacterial blight",
        "how_it_works": "Curcumin and gingerol kill soil-borne pathogens",
        "preparation": "Boil 50g ginger peels + 30g turmeric peels in 1 liter water for 20 minutes. Cool and strain",
        "application": "Dilute 1:2 with water. Drench soil around infected plants twice weekly"
    },
    "Used Coffee Ground Mix": {
        "source": "Coffee or tea waste",
        "target": "Root rot fungi, ants",
        "how_it_works": "Tannins and polyphenols inhibit fungal growth and repel insects",
        "preparation": "Mix dried coffee grounds with equal parts soil. Let it cure for 3-4 days",
        "application": "Apply 2-inch layer around plant base. Refresh monthly"
    },

    # HYDROPONIC SOLUTIONS
    "Lactobacillus Serum (Curd Whey Spray)": {
        "source": "Curd or buttermilk",
        "target": "Root slime, algae, bacterial biofilm",
        "how_it_works": "Beneficial bacteria outcompete pathogens and balance microbiome",
        "preparation": "Mix 100ml fresh curd whey with 900ml dechlorinated water. Let ferment 24 hours",
        "application": "Add 50ml per liter to hydroponic reservoir weekly. Spray on roots during water changes"
    },
    "Citrus Bio-Enzyme": {
        "source": "Citrus peels + jaggery + water",
        "target": "algae, fungal film, Aphids",
        "how_it_works": "Acids and enzymes break down organic buildup and deter pests",
        "preparation": "Mix 100g citrus peels + 30g jaggery in 1 liter water. Ferment 30 days, burp daily",
        "application": "Dilute 1:20 with water. Add to reservoir (10ml/liter) monthly or spray on leaves"
    },
    "Aloe Vera Extract": {
        "source": "Aloe peel or leftover gel",
        "target": "Root rot fungi, leaf disease",
        "how_it_works": "Saponins and salicylic acid act as mild fungicides",
        "preparation": "Blend 50g aloe vera gel/peel with 500ml water. Strain through cloth",
        "application": "Dilute 1:4. Add to reservoir (20ml/liter) bi-weekly or spray on leaves"
    },
    "Hydrogen Peroxide Substitute (Rice Wash Water)": {
        "source": "Fermented rice wash water",
        "target": "deoxygenated root,Root slime",
        "how_it_works": "Gentle lactic acid and beneficial microbes suppress harmful bacteria",
        "preparation": "Collect rice wash water, ferment 3-5 days until slightly sour",
        "application": "Dilute 1:10. Add to reservoir during water changes (50ml/liter)"
    },

    # AEROPONIC SOLUTIONS
    "Diluted Garlic-Citrus Mist": {
        "source": "Garlic + lemon peel extract",
        "target": "Aphids, spider mites",
        "how_it_works": "Volatile oils and acids kill soft-bodied insects on contact",
        "preparation": "Steep 3 crushed garlic cloves + 1 lemon peel in 500ml water for 24 hours. Double strain",
        "application": "Dilute 1:5. Mist roots and leaves every 5 days. Ensure no clogging in nozzles"
    },
    "Lactic Ferment Mist (Curd Whey + Aloe)": {
        "source": "Curd whey + aloe water",
        "target": "Fungal leaf spots, bacterial biofilm",
        "how_it_works": "Lactic bacteria + salicylic acid sterilize surfaces",
        "preparation": "Mix 50ml curd whey + 50ml aloe water + 400ml dechlorinated water",
        "application": "Mist roots 2-3 times weekly. Clean nozzles after each use"
    },
    "Banana Peel Vinegar Extract": {
        "source": "Banana peels fermented in water",
        "target": "Mold",
        "how_it_works": "Acetic acid lowers pH and acts as antifungal agent",
        "preparation": "Ferment 3 banana peels in 500ml water for 7-10 days. Strain thoroughly",
        "application": "Dilute 1:10. Mist affected areas twice weekly. Monitor pH levels"
    },
    "Turmeric-Ginger Atomizer Solution": {
        "source": "Turmeric + ginger peel",
        "target": "Airborne fungus, leaf blight",
        "how_it_works": "Natural antibiotics prevent spore germination",
        "preparation": "Boil 20g turmeric + 20g ginger peel in 1 liter water for 15 minutes. Triple strain when cool",
        "application": "Dilute 1:8. Atomize in grow area 2-3 times weekly. Clean system thoroughly after use"
    }
}

# PEST CONTROL COMBINATIONS BY FARMING TYPE
def generate_pest_control_combinations():
    soil_combinations = [
        ["Garlic-Onion Spray"],
        ["Neem Compost Tea"],
        ["Ginger-Turmeric Extract"],
        ["Used Coffee Ground Mix"]
    ]

    hydro_combinations = [
        ["Lactobacillus Serum (Curd Whey Spray)"],
        ["Citrus Bio-Enzyme"],
        ["Aloe Vera Extract"],
        ["Hydrogen Peroxide Substitute (Rice Wash Water)"]
    ]

    aero_combinations = [
        ["Diluted Garlic-Citrus Mist"],
        ["Lactic Ferment Mist (Curd Whey + Aloe)"],
        ["Banana Peel Vinegar Extract"],
        ["Turmeric-Ginger Atomizer Solution"]
    ]

    return soil_combinations, hydro_combinations, aero_combinations

# LOAD PEST CONTROL IMAGES FROM FOLDER
def load_disease_images_from_folder():
    if not os.path.exists(PEST_DISEASE_FOLDER):
        return []
    files = [f for f in os.listdir(PEST_DISEASE_FOLDER) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.webp'))]
    return [os.path.join(PEST_DISEASE_FOLDER, f) for f in files]

# MATCH PEST/DISEASE TO IMAGE (searches in target pests/diseases) - NEW
def match_disease_to_image(target_pests, all_image_paths):
    """Match pest/disease names from target to images"""
    if not all_image_paths:
        return []

    img_paths = []
    # Split target pests/diseases into individual items
    pests = [p.strip().lower() for p in target_pests.split(',')]

    for pest in pests:
        pest_clean = pest.replace(" ", "").replace("-", "")
        for img_path in all_image_paths:
            img_name = os.path.basename(img_path).lower()
            img_base = os.path.splitext(img_name)[0].replace(" ", "").replace("_", "").replace("-", "")

            # Direct match
            if pest_clean in img_base or img_base in pest_clean:
                if img_path not in img_paths:
                    img_paths.append(img_path)
                    continue

            # Word-by-word match
            pest_words = pest.split()
            for word in pest_words:
                if len(word) > 3 and word in img_base:
                    if img_path not in img_paths:
                        img_paths.append(img_path)
                        break

    return img_paths

# MATCH SOLUTION TO IMAGE (for solution ingredients)
def match_solution_to_image(solution_name, all_image_paths):
    """Match solution name to ingredient images"""
    if not all_image_paths:
        return None

    solution_lower = solution_name.lower().replace(" ", "").replace("_", "").replace("(", "").replace(")", "").replace("-", "")
    for img_path in all_image_paths:
        img_name = os.path.basename(img_path).lower()
        img_base = os.path.splitext(img_name)[0].replace(" ", "").replace("_", "").replace("-", "")
        if solution_lower in img_base or img_base in solution_lower:
            return img_path
        # Also check individual words for matches
        solution_words = solution_name.lower().split()
        for word in solution_words:
            if len(word) > 4 and word in img_base:
                return img_path
    return None

# LOAD SOLUTION INGREDIENT IMAGES FROM FOLDER
def load_pest_images_from_folder():
    if not os.path.exists(PEST_FOLDER):
        return []
    files = [f for f in os.listdir(PEST_FOLDER) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.webp'))]
    return [os.path.join(PEST_FOLDER, f) for f in files]

# GET PEST CONTROL RECOMMENDATIONS - COMPLETELY REWRITTEN
def get_pest_control_recommendations(selected_name):
    farming_type = extract_farming_type(selected_name)
    if not farming_type:
        return (gr.update(visible=False), "", [], None, "") * 4  # 5 outputs per solution

    # Load BOTH disease images AND solution ingredient images
    all_disease_images = load_disease_images_from_folder()
    all_solution_images = load_pest_images_from_folder()

    soil_combos, hydro_combos, aero_combos = generate_pest_control_combinations()
    if farming_type == 'soil':
        selected_combos = soil_combos
    elif farming_type == 'hydro':
        selected_combos = hydro_combos
    else:
        selected_combos = aero_combos

    outputs = []
    for i, combo in enumerate(selected_combos, 1):
        # HEADING + TARGET PESTS (shown first)
        heading_text = f"## 🐛 Pest Control Solution {i}\n\n"

        # Show target pests/diseases first
        for solution in combo:
            if solution in PEST_CONTROL_INSTRUCTIONS:
                inst = PEST_CONTROL_INSTRUCTIONS[solution]
                heading_text += f"### 🎯 Target Pests/Diseases:\n**{inst['target']}**\n\n"

        heading_text += "### Solution:\n"
        for solution in combo:
            heading_text += f"- **{solution}**\n"

        # FIND DISEASE IMAGES (for gallery)
        disease_img_paths = []
        if all_disease_images and combo:
            for solution in combo:
                if solution in PEST_CONTROL_INSTRUCTIONS:
                    target_pests = PEST_CONTROL_INSTRUCTIONS[solution]['target']
                    matched_images = match_disease_to_image(target_pests, all_disease_images)
                    disease_img_paths.extend(matched_images)

        # Remove duplicates
        seen = set()
        disease_img_paths = [x for x in disease_img_paths if not (x in seen or seen.add(x))]

        # Convert to gallery format
        def to_gallery(paths):
            return [(p, os.path.basename(p)) for p in paths if os.path.exists(p)]

        disease_gallery_data = to_gallery(disease_img_paths)

        # FIND SOLUTION IMAGE (single image for solution ingredients)
        solution_img = None
        if all_solution_images and combo:
            for solution in combo:
                img_path = match_solution_to_image(solution, all_solution_images)
                if img_path and os.path.exists(img_path):
                    solution_img = img_path
                    break  # Take first match

        # DETAILED INSTRUCTIONS (no safety tips)
        instruction_text = "### Detailed Information:\n\n"
        for solution in combo:
            if solution in PEST_CONTROL_INSTRUCTIONS:
                inst = PEST_CONTROL_INSTRUCTIONS[solution]
                instruction_text += f"**{solution}:**\n"
                instruction_text += f"- 🌿 Source: {inst['source']}\n"
                instruction_text += f"- 🔬 How It Works: {inst['how_it_works']}\n"
                instruction_text += f"- 🔪 Preparation: {inst['preparation']}\n"
                instruction_text += f"- 💧 Application: {inst['application']}\n\n"

        # Output: [visibility, heading, disease_gallery, solution_image, instructions]
        outputs.extend([gr.update(visible=True), heading_text, disease_gallery_data, solution_img, instruction_text])

    return tuple(outputs)

# ============================================================================
# -------------------- UI LAYOUT WITH TABS -----------------------------------
# ============================================================================

with gr.Blocks(theme=gr.themes.Soft(primary_hue="green")) as app:
    gr.Markdown("# ♻️ Smart Waste Restructure System\nTransform plastic waste into farming containers and discover the perfect plants to grow!")

    with gr.Tabs():
        # TAB 1: RESTRUCTURING
        with gr.Tab("♻️ Restructuring"):
            gr.Markdown("Upload an image of a **plastic waste** item to classify it (bottle/container).\nThen explore eco-friendly **reuse ideas** under Soil, Hydro, and Aeroponics 🌱")
            with gr.Row():
                upload = gr.Image(label="📤 Upload Material", type="filepath", height=160)
                status = gr.Textbox(label="Prediction", interactive=False)
            general_box = gr.Markdown("Upload an image to get started.")
            gr.Markdown("---")
            soil_inst_box = gr.Markdown("")
            soil_gallery = gr.Gallery(columns=6, height=150, label="Soil-based Farming", object_fit="cover", container=False, show_label=True)
            gr.Markdown("---")
            hydro_inst_box = gr.Markdown("")
            hydro_gallery = gr.Gallery(columns=6, height=150, label="Hydroponics", object_fit="cover", container=False, show_label=True)
            gr.Markdown("---")
            aero_inst_box = gr.Markdown("")
            aero_gallery = gr.Gallery(columns=6, height=150, label="Aeroponics", object_fit="cover", container=False, show_label=True)
            gr.Markdown("---")
            with gr.Row():
                choose_btn = gr.Button("Select this design")
                selected_text = gr.Markdown("No selection yet.")
            upload.change(on_upload, inputs=upload, outputs=[status, general_box, soil_inst_box, hydro_inst_box, aero_inst_box, soil_gallery, hydro_gallery, aero_gallery, selected_text])
            soil_gallery.select(on_gallery_select, inputs=[soil_gallery], outputs=[selected_text])
            hydro_gallery.select(on_gallery_select, inputs=[hydro_gallery], outputs=[selected_text])
            aero_gallery.select(on_gallery_select, inputs=[aero_gallery], outputs=[selected_text])
            choose_btn.click(confirm_selected, inputs=selected_text, outputs=selected_text)

        # TAB 2: SEEDLINGS
        with gr.Tab("🌱 Seedlings"):
            gr.Markdown("## Get Personalized Plant Recommendations\nAfter selecting a restructuring design in the previous tab, come here to discover which plants are perfect for your chosen farming method!")
            gr.Markdown("---")
            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("### How to use:\n1. Select a design in the **Restructuring** tab\n2. Come back to this tab\n3. Click \"Show Seedlings\" below\n4. Get 5 recommended plants with images and growing tips!")
                    seedling_btn = gr.Button("🌿 Show Seedlings", variant="primary", size="lg")
            gr.Markdown("---")
            gr.Markdown("# 🌱 Recommended Plants for Your Farming Method")
            plant_blocks = []
            for i in range(5):
                with gr.Column(visible=False) as plant_col:
                    plant_text = gr.Markdown("")
                    plant_img = gr.Image(label=f"Plant {i+1}", height=250, show_label=False)
                    gr.Markdown("---")
                plant_blocks.append((plant_col, plant_text, plant_img))
            all_plant_outputs = []
            for col, text, img in plant_blocks:
                all_plant_outputs.extend([col, text, img])
            seedling_btn.click(generate_seedling_info, inputs=selected_text, outputs=all_plant_outputs)

 # TAB 3: MANURE CREATION (FIXED - HEADING → INGREDIENTS → IMAGES → INSTRUCTIONS)
        with gr.Tab("🍂 Manure Creation"):
            gr.Markdown("## Create Organic Liquid Manure\nBased on your selected farming method, get customized recipes using kitchen waste to create nutrient-rich liquid fertilizer!")
            gr.Markdown("---")
            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("### How to use:\n1. Select a design in the **Restructuring** tab\n2. Come back to this tab\n3. Click \"Show Manure Recipes\" below\n4. Get 4 organic manure recipes tailored to your farming method!")
                    manure_btn = gr.Button("🧪 Show Manure Recipes", variant="primary", size="lg")
            gr.Markdown("---")
            gr.Markdown("# 🧪 Organic Liquid Manure Recipes")
            manure_blocks = []
            for i in range(4):
                with gr.Column(visible=False) as manure_col:
                    # Order: HEADING + INGREDIENTS → IMAGES → DETAILED INSTRUCTIONS
                    manure_heading = gr.Markdown("")  # Recipe heading + ingredients list
                    manure_gallery = gr.Gallery(columns=3, height=200, label=f"Recipe {i+1} Ingredients", object_fit="cover", container=False, show_label=True)
                    manure_instructions = gr.Markdown("")  # Detailed instructions
                    gr.Markdown("---")
                manure_blocks.append((manure_col, manure_heading, manure_gallery, manure_instructions))
            all_manure_outputs = []
            for col, heading, gallery, instructions in manure_blocks:
                all_manure_outputs.extend([col, heading, gallery, instructions])
            manure_btn.click(get_manure_recommendations, inputs=selected_text, outputs=all_manure_outputs)


        # TAB 4: PEST CONTROL
        with gr.Tab("🐛 Pest Control"):
            gr.Markdown("## Natural Pest & Disease Control Solutions\nBased on your selected farming method, get organic pest control recipes using kitchen waste and natural ingredients!")
            gr.Markdown("---")
            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("### How to use:\n1. Select a design in the **Restructuring** tab\n2. Come back to this tab\n3. Click \"Show Pest Control Solutions\" below\n4. Get 4 natural pest control solutions tailored to your farming method!")
                    pest_btn = gr.Button("🐛 Show Pest Control Solutions", variant="primary", size="lg")
            gr.Markdown("---")
            gr.Markdown("# 🐛 Natural Pest & Disease Control Solutions")
            pest_blocks = []
            for i in range(4):
                with gr.Column(visible=False) as pest_col:
                    # Order: HEADING + TARGET → DISEASE GALLERY → SOLUTION IMAGE → INSTRUCTIONS
                    pest_disease_gallery = gr.Gallery(columns=3, height=200, label=f"🎯 Target Pests/Diseases", object_fit="cover", container=False, show_label=True)
                    pest_heading = gr.Markdown("")  # Solution heading + target pests
                    pest_solution_img = gr.Image(label=f"Solution Ingredients", height=200, show_label=True)  # Single solution image (smaller like seedlings)
                    pest_instructions = gr.Markdown("")  # Detailed instructions
                    gr.Markdown("---")
                pest_blocks.append((pest_col, pest_heading, pest_disease_gallery, pest_solution_img, pest_instructions))
            all_pest_outputs = []
            for col, heading, disease_gallery, solution_img, instructions in pest_blocks:
                all_pest_outputs.extend([col, heading, disease_gallery, solution_img, instructions])
            pest_btn.click(get_pest_control_recommendations, inputs=selected_text, outputs=all_pest_outputs)

app.launch(share=True)

✅ Model loaded successfully!


/usr/local/lib/python3.12/dist-packages/gradio/components/base.py:200: UserWarning: show_label has no effect when container is False.
  warnings.warn("show_label has no effect when container is False.")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2534d3929e79e8ca90.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install gradio --quiet
import gradio as gr
import os
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model
import random

# ---------------- CONFIG ----------------
MODEL_PATH = "/content/drive/MyDrive/plastic_classifier_model.h5"
BASE_PATH = "/content/drive/MyDrive/restructure_material"
SEED_FOLDER = "/content/drive/MyDrive/plant_seeds"
MANURE_FOLDER = "/content/drive/MyDrive/manure"  # NEW: Add this folder for manure images
PEST_FOLDER = "/content/drive/MyDrive/pest_control"
PEST_DISEASE_FOLDER = "/content/drive/MyDrive/pest_diseases"# Folder containing pest control ingredient images

# ---------------- MODEL ----------------
try:
    model = load_model(MODEL_PATH, compile=False)
    print("✅ Model loaded successfully!")
except Exception as e:
    print("⚠️ Model load failed, using dummy:", e)
    class Dummy:
        def predict(self, x): return np.array([[0.9, 0.1]])
    model = Dummy()

# ---------------- HELPERS (RESTRUCTURING) ----------------
def classify_uploaded_image(uploaded_path):
    if not uploaded_path:
        return "⚠️ Upload an image first.", None
    img = Image.open(uploaded_path).convert("RGB").resize((160,160))
    arr = np.expand_dims(np.array(img)/255., axis=0)
    pred = model.predict(arr)
    pred = np.array(pred).squeeze()
    if pred.size == 1:
        idx = 0 if pred < 0.5 else 1
        conf = float(pred)*100
    else:
        idx = int(np.argmax(pred))
        conf = float(np.max(pred))*100
    material = "bottle" if idx==0 else "container"
    return f"✅ Predicted Material: {material.upper()}", material

def load_category_images(material):
    folder = os.path.join(BASE_PATH, material.lower())
    if not os.path.exists(folder):
        return [], [], []
    files = [f for f in os.listdir(folder) if f.lower().endswith(('.jpg','.jpeg','.png'))]
    soil = [os.path.join(folder,f) for f in files if 'soil' in f.lower()]
    hydro = [os.path.join(folder,f) for f in files if 'hydro' in f.lower()]
    aero = [os.path.join(folder,f) for f in files if 'aero' in f.lower()]
    return soil, hydro, aero

def get_instructions(material):
    general_instructions = """
### 🧭 General Instructions
- ♻️ Clean the waste material thoroughly before reuse.
- 🪚 Cut or modify safely using proper tools.
- 🌱 Choose a farming method (Soil-based, Hydroponics, Aeroponics).
- 💧 Maintain correct watering or nutrient levels.
- ☀️ Provide adequate sunlight or controlled temperature.
"""
    soil_instructions = """
### 🌱 Soil-based Farming
1. Fill with soil and compost mix.
2. Add sapling and ensure drainage holes.
3. Place under sunlight and water regularly.
"""
    hydro_instructions = """
### 💧 Hydroponics
1. Fill partially with nutrient solution.
2. Support roots using cocopeat or sponge.
3. Maintain aeration and pH balance.
"""
    aero_instructions = """
### 🌬️ Aeroponics
1. Suspend roots in air and mist periodically.
2. Keep spray nozzles clean.
3. Maintain humidity and temperature.
"""
    return general_instructions, soil_instructions, hydro_instructions, aero_instructions

def on_upload(img_path):
    status, material = classify_uploaded_image(img_path)
    general, soil_inst, hydro_inst, aero_inst = get_instructions(material)
    if material is None:
        return status, general, "", "", "", [], [], [], "No selection yet."
    soil, hydro, aero = load_category_images(material)
    def to_gallery(paths):
        return [(p, os.path.basename(p)) for p in paths if os.path.exists(p)]
    return status, general, soil_inst, hydro_inst, aero_inst, to_gallery(soil), to_gallery(hydro), to_gallery(aero), "No selection yet."

def on_gallery_select(evt: gr.SelectData, gallery_data):
    try:
        selected_index = evt.index
        if gallery_data is None or len(gallery_data) == 0:
            return "⚠️ No images in gallery"
        if selected_index >= len(gallery_data):
            return "⚠️ Invalid selection"
        selected_item = gallery_data[selected_index]
        selected_path = selected_item[0]
        selected_name = os.path.basename(selected_path)
        return selected_name
    except Exception as e:
        return f"⚠️ Error: {str(e)}"

def confirm_selected(selected_name):
    if not selected_name or "⚠️" in selected_name or "No selection" in selected_name:
        return "⚠️ No design selected. Click a thumbnail first."
    return f"✅ You selected: **{selected_name}**"

# ============================================================================
# -------------------- SEEDLING FUNCTIONALITY --------------------------------
# ============================================================================

PLANTS = [
    {"name": "Tomato", "soil": True, "hydro": True, "aero": True},
    {"name": "Brinjal", "soil": True, "hydro": True, "aero": False},
    {"name": "Ladies Finger", "soil": True, "hydro": True, "aero": False},
    {"name": "Ridge gaurd", "soil": True, "hydro": False, "aero": False},
    {"name": "Snake gaurd", "soil": True, "hydro": False, "aero": False},
    {"name": "Bottle gaurd", "soil": False, "hydro": True, "aero": False},
    {"name": "Beans", "soil": True, "hydro": True, "aero": False},
    {"name": "Cluster beans", "soil": True, "hydro": False, "aero": False},
    {"name": "Corriander", "soil": True, "hydro": True, "aero": True},
    {"name": "Fenugreek", "soil": True, "hydro": True, "aero": True},
    {"name": "Cucumber", "soil": True, "hydro": False, "aero": True},
    {"name": "Spinach", "soil": False, "hydro": False, "aero": True},
    {"name": "Mint", "soil": False, "hydro": False, "aero": True},
    {"name": "Basil", "soil": True, "hydro": True, "aero": True},
    {"name": "Chilli", "soil": True, "hydro": True, "aero": False},
    {"name": "Lemon grass", "soil": True, "hydro": False, "aero": False},
    {"name": "Aloe vera", "soil": True, "hydro": False, "aero": False},
    {"name": "Lettuce", "soil": False, "hydro": True, "aero": True},
    {"name": "Celery", "soil": False, "hydro": True, "aero": False},
    {"name": "Tomato Cherry", "soil": False, "hydro": False, "aero": False},
    {"name": "Water Spinach", "soil": False, "hydro": True, "aero": True},
    {"name": "Thyme", "soil": False, "hydro": False, "aero": True},
    {"name": "Microgreens", "soil": False, "hydro": False, "aero": True}
]

SEED_PRICES = {
    "Tomato": "₹20-40 per packet", "Brinjal": "₹15-30 per packet",
    "Ladies Finger": "₹20-35 per packet", "Ridge gaurd": "₹25-40 per packet",
    "Snake gaurd": "₹25-40 per packet", "Bottle gaurd": "₹20-35 per packet",
    "Beans": "₹30-50 per packet", "Cluster beans": "₹20-35 per packet",
    "Corriander": "₹10-20 per packet", "Fenugreek": "₹10-20 per packet",
    "Cucumber": "₹20-40 per packet", "Spinach": "₹15-30 per packet",
    "Mint": "₹15-25 per packet", "Basil": "₹20-35 per packet",
    "Chilli": "₹25-45 per packet", "Lemon grass": "₹30-50 per packet",
    "Aloe vera": "₹40-80 per plant", "Lettuce": "₹20-40 per packet",
    "Celery": "₹30-50 per packet", "Water Spinach": "₹15-30 per packet",
    "Thyme": "₹25-45 per packet", "Microgreens": "₹50-100 per packet",
    "Tomato Cherry": "₹30-60 per packet"
}

GROWING_TIPS = {
    "Tomato": "Sow seeds 1cm deep in moist soil; Transplant after 3-4 weeks when seedlings are 6 inches tall",
    "Brinjal": "Start seeds indoors in seedling trays; Transfer to containers after 5-6 weeks",
    "Ladies Finger": "Direct sow seeds 2cm deep in warm soil; Space plants 12 inches apart",
    "Ridge gaurd": "Soak seeds overnight before planting; Provide vertical support for climbing",
    "Snake gaurd": "Plant seeds in nutrient-rich compost; Needs trellis or support structure",
    "Bottle gaurd": "Start in small pots and transplant; Requires strong support for heavy fruits",
    "Beans": "Sow directly 3-4cm deep in well-drained soil; Water regularly but avoid overwatering",
    "Cluster beans": "Plant seeds 2cm deep in rows; Thrives in warm weather with minimal care",
    "Corriander": "Scatter seeds lightly on soil surface; Keep moist until germination in 7-10 days",
    "Fenugreek": "Soak seeds for 12 hours before sowing; Harvest leaves in 3-4 weeks",
    "Cucumber": "Plant 2-3 seeds per hole 2cm deep; Keep soil consistently moist",
    "Spinach": "Sow seeds thinly in shallow furrows; Harvest outer leaves regularly",
    "Mint": "Use stem cuttings or seeds in moist soil; Grows rapidly and spreads quickly",
    "Basil": "Start seeds indoors or in warm spot; Pinch off flowers to promote leaf growth",
    "Chilli": "Start indoors 8-10 weeks before transplanting; Needs warm temperatures to fruit",
    "Lemon grass": "Plant stalks from grocery store in water; Transfer to soil after roots develop",
    "Aloe vera": "Use offsets or pups from mother plant; Needs minimal watering and well-draining soil",
    "Lettuce": "Sow seeds on surface, press lightly; Harvest leaves in 4-6 weeks",
    "Celery": "Start seeds indoors in trays; Transplant after 10-12 weeks",
    "Water Spinach": "Plant stem cuttings in water or moist soil; Grows quickly in warm conditions",
    "Thyme": "Surface sow seeds or use cuttings; Requires well-draining soil and sunlight",
    "Microgreens": "Densely scatter seeds on moist growing medium; Harvest in 7-14 days",
    "Tomato Cherry": "Sow like regular tomatoes but in smaller pots; Provide support for vining varieties"
}

def extract_farming_type(filename):
    if not filename or "No selection" in filename or "⚠️" in filename:
        return None
    filename_lower = filename.lower()
    if 'soil' in filename_lower:
        return 'soil'
    elif 'hydro' in filename_lower:
        return 'hydro'
    elif 'aero' in filename_lower:
        return 'aero'
    return None

def get_suitable_plants(farming_type):
    suitable = []
    for plant in PLANTS:
        if plant.get(farming_type, False):
            suitable.append(plant["name"])
    random.shuffle(suitable)
    return suitable[:5]

def load_seed_images_from_folder():
    if not os.path.exists(SEED_FOLDER):
        return []
    files = [f for f in os.listdir(SEED_FOLDER) if f.lower().endswith(('.jpg', '.jpeg', '.png','.webp'))]
    return [os.path.join(SEED_FOLDER, f) for f in files]

def match_plant_to_image(plant_name, all_image_paths):
    plant_lower = plant_name.lower().replace(" ", "").replace("_", "")
    for img_path in all_image_paths:
        img_name = os.path.basename(img_path).lower()
        img_base = os.path.splitext(img_name)[0].replace(" ", "").replace("_", "")
        if plant_lower in img_base or img_base in plant_lower:
            return img_path
    return None

def generate_seedling_info(selected_name):
    farming_type = extract_farming_type(selected_name)
    if not farming_type:
        return (gr.update(visible=False), "", None) * 5
    all_seed_images = load_seed_images_from_folder()
    if not all_seed_images:
        return (gr.update(visible=True), "⚠️ No seed images found", None) * 5
    plants = get_suitable_plants(farming_type)
    if not plants:
        return (gr.update(visible=True), "No suitable plants found", None) * 5
    outputs = []
    for i in range(5):
        if i < len(plants):
            plant_name = plants[i]
            img_path = match_plant_to_image(plant_name, all_seed_images)
            price = SEED_PRICES.get(plant_name, "₹20-50 per packet")
            tips = GROWING_TIPS.get(plant_name, "Sow seeds in moist soil; Keep in warm, sunny location")
            plant_text = f"## 🌿 {plant_name}\n\n**Growing Instructions:**\n- 🏡 {tips}\n- 💰 Market Price: {price}\n"
            outputs.extend([gr.update(visible=True), plant_text, img_path if img_path and os.path.exists(img_path) else None])
        else:
            outputs.extend([gr.update(visible=False), "", None])
    return tuple(outputs)

# ============================================================================
# -------------------- MANURE CREATION FUNCTIONALITY -------------------------
# ============================================================================

MANURE_INSTRUCTIONS = {
    "Curry leaves": {"prep": "Wash and chop curry leaves into small pieces", "fermentation": "Soak in water for 7-10 days in a covered container", "nutrients": "Rich in nitrogen, iron, and minerals. Excellent for leafy growth", "application": "Dilute 1:5 with water before applying to soil/roots"},
    "Apple cores": {"prep": "Chop apple cores into small pieces, remove seeds", "fermentation": "Soak in water for 5-7 days, stir daily", "nutrients": "Contains potassium, phosphorus, and natural sugars", "application": "Strain and dilute 1:4 with water. Use for flowering plants"},
    "Cucumber peel": {"prep": "Slice cucumber peels thinly for faster decomposition", "fermentation": "Ferment in water for 5-7 days in shade", "nutrients": "High in silica and potassium. Good for stem strength", "application": "Mix 1 part solution with 3 parts water"},
    "Tomato waste": {"prep": "Crush tomato waste to release nutrients quickly", "fermentation": "Steep in water for 7-10 days, covered", "nutrients": "Rich in phosphorus and calcium. Promotes fruiting", "application": "Dilute 1:5 and apply during flowering stage"},
    "Banana peel": {"prep": "Cut banana peels into 1-inch pieces", "fermentation": "Soak in water for 48-72 hours, stir twice daily", "nutrients": "Extremely high in potassium. Boosts flowering and fruiting", "application": "Dilute 1:3 with water. Excellent for all plants"},
    "Potato peel": {"prep": "Chop potato peels finely", "fermentation": "Ferment for 5-7 days in water", "nutrients": "Contains potassium and starch. Good for root development", "application": "Strain and dilute 1:4 before use"},
    "Onion peel": {"prep": "Collect dry outer peels and crush lightly", "fermentation": "Soak in warm water for 24 hours", "nutrients": "Contains sulfur and antioxidants. Natural pest deterrent", "application": "Use undiluted as spray for pest control and nutrition"},
    "Garlic peel": {"prep": "Crush garlic peels to release compounds", "fermentation": "Steep in water for 24-48 hours", "nutrients": "Rich in sulfur. Acts as natural fungicide and pest repellent", "application": "Dilute 1:2 and spray on leaves and soil"},
    "Tea waste": {"prep": "Collect used tea leaves/bags", "fermentation": "Soak in water for 3-5 days", "nutrients": "Contains nitrogen and tannic acid. Lowers pH for acid-loving plants", "application": "Dilute 1:4 with water"},
    "Eggshells": {"prep": "Crush eggshells into fine powder", "fermentation": "Soak in vinegar-water (1:10) for 7-14 days", "nutrients": "Excellent source of calcium. Prevents blossom end rot", "application": "Dilute 1:5 and apply to soil around plants"},
    "Milk waste": {"prep": "Dilute sour milk or curd with water immediately", "fermentation": "Let stand for 24-48 hours", "nutrients": "Rich in calcium, proteins, and beneficial bacteria", "application": "Dilute 1:4 with water. Acts as mild fungicide"},
    "Rice water": {"prep": "Collect water after washing rice (first wash preferred)", "fermentation": "Ferment for 3-5 days until slightly sour", "nutrients": "Contains NPK and promotes beneficial microorganisms", "application": "Dilute 1:2 with water. Excellent for all plants"},
    "Pomegranate peel": {"prep": "Dry peels in sun for 2 days, then powder them", "fermentation": "Soak powder in water for 7-10 days", "nutrients": "High in antioxidants and minerals. Pest deterrent", "application": "Strain and dilute 1:5 before applying"},
    "Mango peel": {"prep": "Chop mango peels into small pieces", "fermentation": "Ferment in water for 7-10 days with occasional stirring", "nutrients": "Rich in vitamins and minerals. Good for overall plant health", "application": "Dilute 1:4 and use during vegetative growth"},
    "Coconut husk": {"prep": "Break coconut husk into small pieces or powder", "fermentation": "Soak in water for 14-21 days (longer fermentation)", "nutrients": "High in potassium and lignin. Improves soil structure", "application": "Dilute 1:3 with water"},
    "Ginger peel waste": {"prep": "Chop ginger peels finely", "fermentation": "Steep in water for 5-7 days", "nutrients": "Contains minerals and natural antibacterial compounds", "application": "Dilute 1:4. Also acts as natural pest repellent"},
    "Cabbage outer leaves": {"prep": "Chop cabbage leaves into small pieces", "fermentation": "Soak in water for 7-10 days", "nutrients": "Rich in nitrogen and calcium. Good for leafy vegetables", "application": "Dilute 1:5 before applying to soil"},
    "Spinach stems": {"prep": "Chop spinach stems finely", "fermentation": "Ferment in water for 5-7 days", "nutrients": "High in iron and nitrogen. Excellent for leafy growth", "application": "Dilute 1:4 with water"}
}

def generate_manure_combinations():
    soil_combinations = [
        ["Banana peel"], ["Rice water"], ["Tea waste"], ["Eggshells"], ["Curry leaves"],
        ["Tomato waste"], ["Apple cores"], ["Milk waste"], ["Onion peel"], ["Garlic peel"]
    ]
    hydro_combinations = [
        ["Banana peel", "Rice water"], ["Tea waste", "Eggshells"], ["Tomato waste", "Milk waste"],
        ["Cucumber peel", "Rice water"], ["Apple cores", "Banana peel"], ["Onion peel", "Garlic peel"],
        ["Curry leaves", "Rice water"], ["Potato peel", "Tea waste"], ["Spinach stems", "Eggshells"], ["Mango peel", "Rice water"]
    ]
    aero_combinations = [
        ["Banana peel", "Rice water", "Tea waste"], ["Eggshells", "Tomato waste", "Milk waste"],
        ["Cucumber peel", "Apple cores", "Rice water"], ["Onion peel", "Garlic peel", "Curry leaves"],
        ["Potato peel", "Banana peel", "Tea waste"], ["Spinach stems", "Cabbage outer leaves", "Rice water"],
        ["Mango peel", "Pomegranate peel", "Rice water"], ["Ginger peel waste", "Garlic peel", "Onion peel"],
        ["Coconut husk", "Banana peel", "Rice water"], ["Apple cores", "Tomato waste", "Eggshells"]
    ]
    return soil_combinations, hydro_combinations, aero_combinations

# NEW FUNCTION: Load manure ingredient images from folder
def load_manure_images_from_folder():
    if not os.path.exists(MANURE_FOLDER):
        return []
    files = [f for f in os.listdir(MANURE_FOLDER) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.webp'))]
    return [os.path.join(MANURE_FOLDER, f) for f in files]

# NEW FUNCTION: Match ingredient name to image (similar to plant matching)
def match_ingredient_to_image(ingredient_name, all_image_paths):
    ingredient_lower = ingredient_name.lower().replace(" ", "").replace("_", "").replace("(", "").replace(")", "").replace(",", "")
    for img_path in all_image_paths:
        img_name = os.path.basename(img_path).lower()
        img_base = os.path.splitext(img_name)[0].replace(" ", "").replace("_", "")
        if ingredient_lower in img_base or img_base in ingredient_lower:
            return img_path
    return None

# UPDATED FUNCTION: Get manure recommendations with images (FIXED VERSION - LIKE RESTRUCTURING)
# UPDATED FUNCTION: Get manure recommendations with images (FIXED VERSION - LIKE RESTRUCTURING)
def get_manure_recommendations(selected_name):
    farming_type = extract_farming_type(selected_name)
    if not farming_type:
        return (gr.update(visible=False), "", [], "") * 4  # 4 recipes × 4 outputs each (visibility, heading, gallery, instructions)

    # Load all manure ingredient images
    all_manure_images = load_manure_images_from_folder()

    soil_combos, hydro_combos, aero_combos = generate_manure_combinations()
    if farming_type == 'soil':
        selected_combos = random.sample(soil_combos, 4)
    elif farming_type == 'hydro':
        selected_combos = random.sample(hydro_combos, 4)
    else:
        selected_combos = random.sample(aero_combos, 4)

    outputs = []
    for i, combo in enumerate(selected_combos, 1):
        # HEADING + INGREDIENTS
        heading_text = f"## 🧪 Liquid Manure Recipe {i}\n\n### Ingredients:\n"
        for ingredient in combo:
            heading_text += f"- **{ingredient}**\n"

        # DETAILED INSTRUCTIONS (separate from heading)
        instruction_text = "### Detailed Instructions:\n\n"
        for ingredient in combo:
            if ingredient in MANURE_INSTRUCTIONS:
                inst = MANURE_INSTRUCTIONS[ingredient]
                instruction_text += f"**{ingredient}:**\n- 🔪 Preparation: {inst['prep']}\n- ⏱️ Fermentation: {inst['fermentation']}\n- 🌱 Nutrients: {inst['nutrients']}\n- 💧 Application: {inst['application']}\n\n"

        if len(combo) == 1:
            instruction_text += "### Final Steps:\n1. Use a clean plastic container for fermentation\n2. Cover with a breathable cloth to allow gases to escape\n3. Stir once daily during fermentation\n4. Strain before use\n5. Store in a cool, dark place for up to 2 weeks\n"
        elif len(combo) == 2:
            instruction_text += "### Final Steps:\n1. Prepare both ingredients separately as per instructions above\n2. After individual fermentation, mix the two solutions in equal parts\n3. Let the mixture settle for 24 hours\n4. Dilute the final mixture 1:5 with water before application\n5. Use within 1 week for best results\n"
        else:
            instruction_text += "### Final Steps:\n1. Prepare all three ingredients separately as per instructions above\n2. After fermentation, combine all three solutions in equal parts\n3. Mix thoroughly and let it rest for 48 hours\n4. Dilute the final mixture 1:6 with water before application\n5. For aeroponics, ensure the solution is well-strained to prevent nozzle clogging\n6. Use within 5 days for optimal nutrient content\n"

        # FIXED: Find images for ALL ingredients in the combo (like restructuring part)
        img_paths = []
        if all_manure_images and combo:
            for ingredient in combo:
                img_path = match_ingredient_to_image(ingredient, all_manure_images)
                if img_path and os.path.exists(img_path):
                    img_paths.append(img_path)

        # Convert to gallery format (like in restructuring: to_gallery function)
        def to_gallery(paths):
            return [(p, os.path.basename(p)) for p in paths if os.path.exists(p)]

        gallery_data = to_gallery(img_paths)

        # Output: [visibility, heading_with_ingredients, gallery, detailed_instructions]
        outputs.extend([gr.update(visible=True), heading_text, gallery_data, instruction_text])

    return tuple(outputs)
def load_pest_images_from_folder():
    if not os.path.exists(PEST_FOLDER):
        return []
    files = [f for f in os.listdir(PEST_FOLDER) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.webp'))]
    return [os.path.join(PEST_FOLDER, f) for f in files]

# MATCH PEST CONTROL SOLUTION TO IMAGE
def match_pest_to_image(solution_name, all_image_paths):
    solution_lower = solution_name.lower().replace(" ", "").replace("_", "").replace("(", "").replace(")", "").replace("-", "")
    for img_path in all_image_paths:
        img_name = os.path.basename(img_path).lower()
        img_base = os.path.splitext(img_name)[0].replace(" ", "").replace("_", "").replace("-", "")
        if solution_lower in img_base or img_base in solution_lower:
            return img_path
        # Also check individual words for matches
        solution_words = solution_name.lower().split()
        for word in solution_words:
            if len(word) > 3 and word in img_base:
                return img_path
    return None

# PEST CONTROL INSTRUCTIONS DICTIONARY
PEST_CONTROL_INSTRUCTIONS = {
    # SOIL-BASED SOLUTIONS
    "Garlic-Onion Spray": {
        "source": "Garlic, onion peels, chili",
        "target": "Aphids, beetles, leaf spot fungus",
        "how_it_works": "Allicin and sulfur act as natural antibiotics and repellents",
        "preparation": "Crush 5-6 garlic cloves, 1 onion, and 2 chilies. Steep in 1 liter water for 24 hours",
        "application": "Strain and spray on affected leaves every 3-4 days. Best applied in early morning"
    },
    "Neem Compost Tea": {
        "source": "Neem seeds + kitchen compost",
        "target": "Aphids",
        "how_it_works": "Azadirachtin in neem regulates pest growth and disrupts feeding",
        "preparation": "Soak 100g crushed neem seeds + 200g compost in 2 liters water for 48 hours",
        "application": "Dilute 1:3 with water. Apply to soil around plant base weekly"
    },
    "Ginger-Turmeric Extract": {
        "source": "Ginger & turmeric peels",
        "target": "Fungal wilt, bacterial blight",
        "how_it_works": "Curcumin and gingerol kill soil-borne pathogens",
        "preparation": "Boil 50g ginger peels + 30g turmeric peels in 1 liter water for 20 minutes. Cool and strain",
        "application": "Dilute 1:2 with water. Drench soil around infected plants twice weekly"
    },
    "Used Coffee Ground Mix": {
        "source": "Coffee or tea waste",
        "target": "Root rot fungi, ants",
        "how_it_works": "Tannins and polyphenols inhibit fungal growth and repel insects",
        "preparation": "Mix dried coffee grounds with equal parts soil. Let it cure for 3-4 days",
        "application": "Apply 2-inch layer around plant base. Refresh monthly"
    },

    # HYDROPONIC SOLUTIONS
    "Lactobacillus Serum (Curd Whey Spray)": {
        "source": "Curd or buttermilk",
        "target": "Root slime, algae, bacterial biofilm",
        "how_it_works": "Beneficial bacteria outcompete pathogens and balance microbiome",
        "preparation": "Mix 100ml fresh curd whey with 900ml dechlorinated water. Let ferment 24 hours",
        "application": "Add 50ml per liter to hydroponic reservoir weekly. Spray on roots during water changes"
    },
    "Citrus Bio-Enzyme": {
        "source": "Citrus peels + jaggery + water",
        "target": "algae, fungal film, Aphids",
        "how_it_works": "Acids and enzymes break down organic buildup and deter pests",
        "preparation": "Mix 100g citrus peels + 30g jaggery in 1 liter water. Ferment 30 days, burp daily",
        "application": "Dilute 1:20 with water. Add to reservoir (10ml/liter) monthly or spray on leaves"
    },
    "Aloe Vera Extract": {
        "source": "Aloe peel or leftover gel",
        "target": "Root rot fungi, leaf disease",
        "how_it_works": "Saponins and salicylic acid act as mild fungicides",
        "preparation": "Blend 50g aloe vera gel/peel with 500ml water. Strain through cloth",
        "application": "Dilute 1:4. Add to reservoir (20ml/liter) bi-weekly or spray on leaves"
    },
    "Hydrogen Peroxide Substitute (Rice Wash Water)": {
        "source": "Fermented rice wash water",
        "target": "deoxygenated root,Root slime",
        "how_it_works": "Gentle lactic acid and beneficial microbes suppress harmful bacteria",
        "preparation": "Collect rice wash water, ferment 3-5 days until slightly sour",
        "application": "Dilute 1:10. Add to reservoir during water changes (50ml/liter)"
    },

    # AEROPONIC SOLUTIONS
    "Diluted Garlic-Citrus Mist": {
        "source": "Garlic + lemon peel extract",
        "target": "Aphids, spider mites",
        "how_it_works": "Volatile oils and acids kill soft-bodied insects on contact",
        "preparation": "Steep 3 crushed garlic cloves + 1 lemon peel in 500ml water for 24 hours. Double strain",
        "application": "Dilute 1:5. Mist roots and leaves every 5 days. Ensure no clogging in nozzles"
    },
    "Lactic Ferment Mist (Curd Whey + Aloe)": {
        "source": "Curd whey + aloe water",
        "target": "Fungal leaf spots, bacterial biofilm",
        "how_it_works": "Lactic bacteria + salicylic acid sterilize surfaces",
        "preparation": "Mix 50ml curd whey + 50ml aloe water + 400ml dechlorinated water",
        "application": "Mist roots 2-3 times weekly. Clean nozzles after each use"
    },
    "Banana Peel Vinegar Extract": {
        "source": "Banana peels fermented in water",
        "target": "Mold",
        "how_it_works": "Acetic acid lowers pH and acts as antifungal agent",
        "preparation": "Ferment 3 banana peels in 500ml water for 7-10 days. Strain thoroughly",
        "application": "Dilute 1:10. Mist affected areas twice weekly. Monitor pH levels"
    },
    "Turmeric-Ginger Atomizer Solution": {
        "source": "Turmeric + ginger peel",
        "target": "Airborne fungus, leaf blight",
        "how_it_works": "Natural antibiotics prevent spore germination",
        "preparation": "Boil 20g turmeric + 20g ginger peel in 1 liter water for 15 minutes. Triple strain when cool",
        "application": "Dilute 1:8. Atomize in grow area 2-3 times weekly. Clean system thoroughly after use"
    }
}

# PEST CONTROL COMBINATIONS BY FARMING TYPE
def generate_pest_control_combinations():
    soil_combinations = [
        ["Garlic-Onion Spray"],
        ["Neem Compost Tea"],
        ["Ginger-Turmeric Extract"],
        ["Used Coffee Ground Mix"]
    ]

    hydro_combinations = [
        ["Lactobacillus Serum (Curd Whey Spray)"],
        ["Citrus Bio-Enzyme"],
        ["Aloe Vera Extract"],
        ["Hydrogen Peroxide Substitute (Rice Wash Water)"]
    ]

    aero_combinations = [
        ["Diluted Garlic-Citrus Mist"],
        ["Lactic Ferment Mist (Curd Whey + Aloe)"],
        ["Banana Peel Vinegar Extract"],
        ["Turmeric-Ginger Atomizer Solution"]
    ]

    return soil_combinations, hydro_combinations, aero_combinations

# LOAD PEST CONTROL IMAGES FROM FOLDER
def load_disease_images_from_folder():
    if not os.path.exists(PEST_DISEASE_FOLDER):
        return []
    files = [f for f in os.listdir(PEST_DISEASE_FOLDER) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.webp'))]
    return [os.path.join(PEST_DISEASE_FOLDER, f) for f in files]

# MATCH PEST/DISEASE TO IMAGE (searches in target pests/diseases) - NEW
def match_disease_to_image(target_pests, all_image_paths):
    """Match pest/disease names from target to images"""
    if not all_image_paths:
        return []

    img_paths = []
    # Split target pests/diseases into individual items
    pests = [p.strip().lower() for p in target_pests.split(',')]

    for pest in pests:
        pest_clean = pest.replace(" ", "").replace("-", "")
        for img_path in all_image_paths:
            img_name = os.path.basename(img_path).lower()
            img_base = os.path.splitext(img_name)[0].replace(" ", "").replace("_", "").replace("-", "")

            # Direct match
            if pest_clean in img_base or img_base in pest_clean:
                if img_path not in img_paths:
                    img_paths.append(img_path)
                    continue

            # Word-by-word match
            pest_words = pest.split()
            for word in pest_words:
                if len(word) > 3 and word in img_base:
                    if img_path not in img_paths:
                        img_paths.append(img_path)
                        break

    return img_paths

# MATCH SOLUTION TO IMAGE (for solution ingredients)
def match_solution_to_image(solution_name, all_image_paths):
    """Match solution name to ingredient images"""
    if not all_image_paths:
        return None

    solution_lower = solution_name.lower().replace(" ", "").replace("_", "").replace("(", "").replace(")", "").replace("-", "")
    for img_path in all_image_paths:
        img_name = os.path.basename(img_path).lower()
        img_base = os.path.splitext(img_name)[0].replace(" ", "").replace("_", "").replace("-", "")
        if solution_lower in img_base or img_base in solution_lower:
            return img_path
        # Also check individual words for matches
        solution_words = solution_name.lower().split()
        for word in solution_words:
            if len(word) > 4 and word in img_base:
                return img_path
    return None

# LOAD SOLUTION INGREDIENT IMAGES FROM FOLDER
def load_pest_images_from_folder():
    if not os.path.exists(PEST_FOLDER):
        return []
    files = [f for f in os.listdir(PEST_FOLDER) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.webp'))]
    return [os.path.join(PEST_FOLDER, f) for f in files]

# GET PEST CONTROL RECOMMENDATIONS - COMPLETELY REWRITTEN
def get_pest_control_recommendations(selected_name):
    farming_type = extract_farming_type(selected_name)
    if not farming_type:
        return (gr.update(visible=False), "", [], None, "") * 4  # 5 outputs per solution

    # Load BOTH disease images AND solution ingredient images
    all_disease_images = load_disease_images_from_folder()
    all_solution_images = load_pest_images_from_folder()

    soil_combos, hydro_combos, aero_combos = generate_pest_control_combinations()
    if farming_type == 'soil':
        selected_combos = soil_combos
    elif farming_type == 'hydro':
        selected_combos = hydro_combos
    else:
        selected_combos = aero_combos

    outputs = []
    for i, combo in enumerate(selected_combos, 1):
        # HEADING + TARGET PESTS (shown first)
        heading_text = f"## 🐛 Pest Control Solution {i}\n\n"

        # Show target pests/diseases first
        for solution in combo:
            if solution in PEST_CONTROL_INSTRUCTIONS:
                inst = PEST_CONTROL_INSTRUCTIONS[solution]
                heading_text += f"### 🎯 Target Pests/Diseases:\n**{inst['target']}**\n\n"

        heading_text += "### Solution:\n"
        for solution in combo:
            heading_text += f"- **{solution}**\n"

        # FIND DISEASE IMAGES (for gallery)
        disease_img_paths = []
        if all_disease_images and combo:
            for solution in combo:
                if solution in PEST_CONTROL_INSTRUCTIONS:
                    target_pests = PEST_CONTROL_INSTRUCTIONS[solution]['target']
                    matched_images = match_disease_to_image(target_pests, all_disease_images)
                    disease_img_paths.extend(matched_images)

        # Remove duplicates
        seen = set()
        disease_img_paths = [x for x in disease_img_paths if not (x in seen or seen.add(x))]

        # Convert to gallery format
        def to_gallery(paths):
            return [(p, os.path.basename(p)) for p in paths if os.path.exists(p)]

        disease_gallery_data = to_gallery(disease_img_paths)

        # FIND SOLUTION IMAGE (single image for solution ingredients)
        solution_img = None
        if all_solution_images and combo:
            for solution in combo:
                img_path = match_solution_to_image(solution, all_solution_images)
                if img_path and os.path.exists(img_path):
                    solution_img = img_path
                    break  # Take first match

        # DETAILED INSTRUCTIONS (no safety tips)
        instruction_text = "### Detailed Information:\n\n"
        for solution in combo:
            if solution in PEST_CONTROL_INSTRUCTIONS:
                inst = PEST_CONTROL_INSTRUCTIONS[solution]
                instruction_text += f"**{solution}:**\n"
                instruction_text += f"- 🌿 Source: {inst['source']}\n"
                instruction_text += f"- 🔬 How It Works: {inst['how_it_works']}\n"
                instruction_text += f"- 🔪 Preparation: {inst['preparation']}\n"
                instruction_text += f"- 💧 Application: {inst['application']}\n\n"

        # Output: [visibility, heading, disease_gallery, solution_image, instructions]
        outputs.extend([gr.update(visible=True), heading_text, disease_gallery_data, solution_img, instruction_text])

    return tuple(outputs)

# ============================================================================
# -------------------- UI LAYOUT WITH TABS -----------------------------------
# ============================================================================
# ============================================================================
# -------------------- UI LAYOUT WITH TABS -----------------------------------
# ============================================================================

with gr.Blocks(theme=gr.themes.Soft(primary_hue="green"), title="Sprout Smart") as app:
    # MODIFIED: Centered title with larger font
    gr.Markdown(
        """
        <h1 style='text-align: center; font-size: 48px; font-weight: bold; color: #2d5016; margin-bottom: 10px;'>
        ♻️ Sprout Smart
        </h1>
        <p style='text-align: center; font-size: 18px; color: #5a7c3e;'>
        Transform plastic waste into farming containers and discover the perfect plants to grow!
        </p>
        """,
        elem_id="main-title"
    )
    with gr.Tabs():
        # TAB 1: RESTRUCTURING - MODIFIED with larger, attractive styling
        with gr.Tab("♻️ Restructuring", elem_id="tab-restructure"):
            gr.Markdown(
                """
                <h2 style='font-size: 32px; font-weight: bold; color: #2d5016;'>
                ♻️ Restructuring
                </h2>
                <p style='font-size: 16px;'>
                Upload an image of a <strong>plastic waste</strong> item to classify it (bottle/container).<br>
                Then explore eco-friendly <strong>reuse ideas</strong> under Soil, Hydro, and Aeroponics 🌱
                </p>
                """
            )
            with gr.Row():
                upload = gr.Image(label="📤 Upload Material", type="filepath", height=160)
                status = gr.Textbox(label="Prediction", interactive=False)
            general_box = gr.Markdown("Upload an image to get started.")
            gr.Markdown("---")
            soil_inst_box = gr.Markdown("")
            soil_gallery = gr.Gallery(columns=6, height=150, label="Soil-based Farming", object_fit="cover", container=False, show_label=True)
            gr.Markdown("---")
            hydro_inst_box = gr.Markdown("")
            hydro_gallery = gr.Gallery(columns=6, height=150, label="Hydroponics", object_fit="cover", container=False, show_label=True)
            gr.Markdown("---")
            aero_inst_box = gr.Markdown("")
            aero_gallery = gr.Gallery(columns=6, height=150, label="Aeroponics", object_fit="cover", container=False, show_label=True)
            gr.Markdown("---")
            with gr.Row():
                choose_btn = gr.Button("Select this design")
                selected_text = gr.Markdown("No selection yet.")
            upload.change(on_upload, inputs=upload, outputs=[status, general_box, soil_inst_box, hydro_inst_box, aero_inst_box, soil_gallery, hydro_gallery, aero_gallery, selected_text])
            soil_gallery.select(on_gallery_select, inputs=[soil_gallery], outputs=[selected_text])
            hydro_gallery.select(on_gallery_select, inputs=[hydro_gallery], outputs=[selected_text])
            aero_gallery.select(on_gallery_select, inputs=[aero_gallery], outputs=[selected_text])
            choose_btn.click(confirm_selected, inputs=selected_text, outputs=selected_text)

        # TAB 2: SEEDLINGS - MODIFIED with larger, attractive styling
        with gr.Tab("🌱 Seedlings", elem_id="tab-seedlings"):
            gr.Markdown(
                """
                <h2 style='font-size: 32px; font-weight: bold; color: #2d5016;'>
                🌱 Seedlings
                </h2>
                <p style='font-size: 16px;'>
                Get Personalized Plant Recommendations
                </p>
                """
            )
            gr.Markdown("After selecting a restructuring design in the previous tab, come here to discover which plants are perfect for your chosen farming method!")
            gr.Markdown("---")
            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("### How to use:\n1. Select a design in the **Restructuring** tab\n2. Come back to this tab\n3. Click \"Show Seedlings\" below\n4. Get 5 recommended plants with images and growing tips!")
                    seedling_btn = gr.Button("🌿 Show Seedlings", variant="primary", size="lg")
            gr.Markdown("---")
            gr.Markdown("# 🌱 Recommended Plants for Your Farming Method")
            plant_blocks = []
            for i in range(5):
                with gr.Column(visible=False) as plant_col:
                    plant_text = gr.Markdown("")
                    plant_img = gr.Image(label=f"Plant {i+1}", height=250, show_label=False)
                    gr.Markdown("---")
                plant_blocks.append((plant_col, plant_text, plant_img))
            all_plant_outputs = []
            for col, text, img in plant_blocks:
                all_plant_outputs.extend([col, text, img])
            seedling_btn.click(generate_seedling_info, inputs=selected_text, outputs=all_plant_outputs)

        # TAB 3: MANURE CREATION - MODIFIED with larger, attractive styling
        with gr.Tab("🍂 Manure Creation", elem_id="tab-manure"):
            gr.Markdown(
                """
                <h2 style='font-size: 32px; font-weight: bold; color: #2d5016;'>
                🍂 Manure Creation
                </h2>
                <p style='font-size: 16px;'>
                Create Organic Liquid Manure
                </p>
                """
            )
            gr.Markdown("Based on your selected farming method, get customized recipes using kitchen waste to create nutrient-rich liquid fertilizer!")
            gr.Markdown("---")
            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("### How to use:\n1. Select a design in the **Restructuring** tab\n2. Come back to this tab\n3. Click \"Show Manure Recipes\" below\n4. Get 4 organic manure recipes tailored to your farming method!")
                    manure_btn = gr.Button("🧪 Show Manure Recipes", variant="primary", size="lg")
            gr.Markdown("---")
            gr.Markdown("# 🧪 Organic Liquid Manure Recipes")
            manure_blocks = []
            for i in range(4):
                with gr.Column(visible=False) as manure_col:
                    manure_heading = gr.Markdown("")
                    manure_gallery = gr.Gallery(columns=3, height=200, label=f"Recipe {i+1} Ingredients", object_fit="cover", container=False, show_label=True)
                    manure_instructions = gr.Markdown("")
                    gr.Markdown("---")
                manure_blocks.append((manure_col, manure_heading, manure_gallery, manure_instructions))
            all_manure_outputs = []
            for col, heading, gallery, instructions in manure_blocks:
                all_manure_outputs.extend([col, heading, gallery, instructions])
            manure_btn.click(get_manure_recommendations, inputs=selected_text, outputs=all_manure_outputs)

        # TAB 4: PEST CONTROL - MODIFIED with larger, attractive styling
        with gr.Tab("🐛 Pest Control", elem_id="tab-pest"):
            gr.Markdown(
                """
                <h2 style='font-size: 32px; font-weight: bold; color: #2d5016;'>
                🐛 Pest Control
                </h2>
                <p style='font-size: 16px;'>
                Natural Pest & Disease Control Solutions
                </p>
                """
            )
            gr.Markdown("Based on your selected farming method, get organic pest control recipes using kitchen waste and natural ingredients!")
            gr.Markdown("---")
            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("### How to use:\n1. Select a design in the **Restructuring** tab\n2. Come back to this tab\n3. Click \"Show Pest Control Solutions\" below\n4. Get 4 natural pest control solutions tailored to your farming method!")
                    pest_btn = gr.Button("🐛 Show Pest Control Solutions", variant="primary", size="lg")
            gr.Markdown("---")
            gr.Markdown("# 🐛 Natural Pest & Disease Control Solutions")
            pest_blocks = []
            for i in range(4):
                with gr.Column(visible=False) as pest_col:
                    pest_disease_gallery = gr.Gallery(columns=3, height=200, label=f"🎯 Target Pests/Diseases", object_fit="cover", container=False, show_label=True)
                    pest_heading = gr.Markdown("")
                    pest_solution_img = gr.Image(label=f"Solution Ingredients", height=200, show_label=True)
                    pest_instructions = gr.Markdown("")
                    gr.Markdown("---")
                pest_blocks.append((pest_col, pest_heading, pest_disease_gallery, pest_solution_img, pest_instructions))
            all_pest_outputs = []
            for col, heading, disease_gallery, solution_img, instructions in pest_blocks:
                all_pest_outputs.extend([col, heading, disease_gallery, solution_img, instructions])
            pest_btn.click(get_pest_control_recommendations, inputs=selected_text, outputs=all_pest_outputs)
                    # ==================== TAB 3: COMMUNITY ====================
        with gr.Tab("👥 Community"):
            gr.Markdown("<br><br><br>")  # Add spacing from top

            with gr.Row():
                with gr.Column(scale=1):
                    pass  # Empty column for centering
            with gr.Column(scale=1):
                connect_btn = gr.Button("🤝 Connect with People", variant="primary", size="lg")
                community_message = gr.Markdown("", elem_id="community_msg")
            with gr.Column(scale=1):
                pass  # Empty column for centering

           # Wiring for community functionality
            def show_no_users():
                return "### ⚠️ No users found\n\nThere are currently no users in the community. Be the first to join!"

            connect_btn.click(
                show_no_users,
                inputs=None,
                outputs=community_message
    )

app.launch(share=True)

✅ Model loaded successfully!


/usr/local/lib/python3.12/dist-packages/gradio/components/base.py:200: UserWarning: show_label has no effect when container is False.
  warnings.warn("show_label has no effect when container is False.")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e0e9e8abf1e4b83e9e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
